## <font face="宋体" size=5 color=#A52A2A>超参数搜索
- <font face="宋体" size=3 color=#A52A2A>网格搜索+随机搜索

In [ ]:
import keras_tuner as kt
from keras.models import Sequential
from keras.layers import LSTM, Dense, Dropout, Masking
from keras.optimizers import Adam
from keras.callbacks import LearningRateScheduler, LambdaCallback
import keras.backend as K

# 学习率调整函数
def scheduler(epoch):
    # 每隔5个epoch，学习率减小为原来的1/5
    if epoch % 5 == 0 and epoch != 0:
        lr = K.get_value(model.optimizer.lr)
        if lr > 1e-5:
            K.set_value(model.optimizer.lr, lr * 0.2)
            print("lr changed to {}".format(lr * 0.2))
    return K.get_value(model.optimizer.lr)

reduce_lr = LearningRateScheduler(scheduler)

# LSTM模型构建函数
def build_model(hp):
    model = Sequential()
    model.add(Masking(mask_value=0, input_shape=(TIME_STEPS, INPUT_DIMS)))
    model.add(LSTM(units=hp.Int('units', min_value=32, max_value=128, step=4),
                   return_sequences=True))
    model.add(Dropout(rate=hp.Float('dropout_1', min_value=0, max_value=0.8, step=0.05)))
    model.add(LSTM(units=hp.Int('units_l2', min_value=16, max_value=64, step=4),
                   return_sequences=False))
    model.add(Dropout(rate=hp.Float('dropout_2', min_value=0, max_value=0.8, step=0.05)))
    model.add(Dense(1))

    # 设置初始学习率
    initial_learning_rate = 1e-3
    model.compile(optimizer=Adam(learning_rate=initial_learning_rate), loss='mae', metrics=['mse', 'mae'])
    return model

# 超参数搜索和训练函数
def combined_search_tuning(x_train, y_train):
    # 随机搜索配置
    tuner_random = kt.RandomSearch(
        build_model,
        objective='val_loss',
        max_trials=20,
        executions_per_trial=2,
        directory='my_dir',
        project_name='lstm_random_search'
    )

    # 进行随机搜索
    tuner_random.search(x_train, y_train, epochs=20, validation_split=0.2, callbacks=[reduce_lr])
    best_hps_random = tuner_random.get_best_hyperparameters(num_trials=1)[0]

    # 网格搜索范围
    param_grid = {
        'units': [best_hps_random.get('units') - 32, best_hps_random.get('units'), best_hps_random.get('units') + 32],
        'dropout_1': [best_hps_random.get('dropout_1') - 0.1, best_hps_random.get('dropout_1'), best_hps_random.get('dropout_1') + 0.1],
        'dropout_2': [best_hps_random.get('dropout_2') - 0.1, best_hps_random.get('dropout_2'), best_hps_random.get('dropout_2') + 0.1],
    }

    # 网格搜索配置
    tuner_grid = kt.Hyperband(
        build_model,
        hyperparameters=kt.HyperParameters().Fixed(param_grid),
        objective='val_loss',
        max_epochs=20,
        factor=2,
        directory='my_dir',
        project_name='lstm_grid_search'
    )

    # 进行网格搜索
    tuner_grid.search(x_train, y_train, epochs=20, validation_split=0.2, callbacks=[reduce_lr])
    return tuner_grid.get_best_hyperparameters(num_trials=1)[0]

# 获取最优超参数
best_hps_combined = combined_search_tuning(x_train_rainy, y_train_rainy)

# 打印最优超参数
best_hyperparameters = {
    'units': best_hps_combined.get('units'),
    'dropout_1': best_hps_combined.get('dropout_1'),
    'dropout_2': best_hps_combined.get('dropout_2'),
}
print("最优超参数：", best_hyperparameters)

TIME_STEPS = x_train_rainy.shape[1]
INPUT_DIMS = x_train_rainy.shape[2]

# 训练模型时应用学习率调整策略
model = build_model(best_hps_combined)
lr_scheduler = LambdaCallback(on_epoch_begin=lambda epoch, logs: scheduler(epoch, K.get_value(model.optimizer.lr), model))
history = model.fit(x_train_rainy, y_train_rainy, epochs=100, batch_size=64, validation_split=0.1, callbacks=[lr_scheduler])

#### 学习率策略不变

In [ ]:
import keras_tuner as kt
from keras.models import Sequential
from keras.layers import LSTM, Dense, Dropout, Masking
from keras.optimizers import Adam


# LSTM模型构建函数
def build_model(hp):
    model = Sequential()
    model.add(Masking(mask_value=0, input_shape=(TIME_STEPS, INPUT_DIMS)))
    model.add(LSTM(units=hp.Int('units', min_value=32, max_value=128, step=4),
                   return_sequences=True))
    model.add(Dropout(rate=hp.Float('dropout_1', min_value=0, max_value=0.8, step=0.05)))
    model.add(LSTM(units=hp.Int('units_l2', min_value=16, max_value=64, step=4),
                   return_sequences=False))
    model.add(Dropout(rate=hp.Float('dropout_2', min_value=0, max_value=0.8, step=0.05)))
    model.add(Dense(1))

    # 设置初始学习率
    initial_learning_rate = 1e-4
    model.compile(optimizer=Adam(learning_rate=initial_learning_rate), loss='mae', metrics=['mse', 'mae'])
    return model


# 超参数搜索和训练函数
def combined_search_tuning(x_train, y_train):
    # 随机搜索配置
    tuner_random = kt.RandomSearch(
        build_model,
        objective='val_loss',
        max_trials=30,
        executions_per_trial=2,
        directory='my_dir',
        project_name='lstm_random_search'
    )

    # 进行随机搜索
    tuner_random.search(x_train, y_train, epochs=15, validation_split=0.2)
    best_hps_random = tuner_random.get_best_hyperparameters(num_trials=1)[0]

    # 网格搜索范围
    param_grid = {
        'units': [best_hps_random.get('units') - 32, best_hps_random.get('units'), best_hps_random.get('units') + 32],
        'dropout_1': [best_hps_random.get('dropout_1') - 0.1, best_hps_random.get('dropout_1'), best_hps_random.get('dropout_1') + 0.1],
        'dropout_2': [best_hps_random.get('dropout_2') - 0.1, best_hps_random.get('dropout_2'), best_hps_random.get('dropout_2') + 0.1],
    }

    # 网格搜索配置
    tuner_grid = kt.Hyperband(
        build_model,
        hyperparameters=kt.HyperParameters().Fixed(param_grid),
        objective='val_loss',
        max_epochs=50,
        factor=2,
        directory='my_dir',
        project_name='lstm_grid_search'
    )

    # 进行网格搜索
    tuner_grid.search(x_train, y_train, epochs=15, validation_split=0.2)
    return tuner_grid.get_best_hyperparameters(num_trials=1)[0]

# 获取最优超参数
best_hps_combined = combined_search_tuning(x_train_rainy, y_train_rainy)

# 打印最优超参数
best_hyperparameters = {
    'units': best_hps_combined.get('units'),
    'dropout_1': best_hps_combined.get('dropout_1'),
    'dropout_2': best_hps_combined.get('dropout_2'),
}
print("最优超参数：", best_hyperparameters)

TIME_STEPS = x_train_rainy.shape[1]
INPUT_DIMS = x_train_rainy.shape[2]

# 训练模型
model = build_model(best_hps_combined)
history = model.fit(x_train_rainy, y_train_rainy, epochs=100, batch_size=64, validation_split=0.2)

### 细化搜索

In [ ]:
import keras_tuner as kt
from keras.models import Sequential
from keras.layers import LSTM, Dense, Dropout, Masking
from keras.optimizers import Adam

TIME_STEPS = x_train_rainy.shape[1]
INPUT_DIMS = x_train_rainy.shape[2]
# LSTM模型构建函数
def build_model(hp):
    model = Sequential()
    model.add(Masking(mask_value=0, input_shape=(TIME_STEPS, INPUT_DIMS)))
    model.add(LSTM(units=hp.Int('units', min_value=96, max_value=256, step=4),
                   return_sequences=True))
    model.add(Dropout(rate=hp.Float('dropout_1', min_value=0, max_value=0.5, step=0.05)))
    model.add(LSTM(units=hp.Int('units_l2', min_value=48, max_value=128, step=4),
                   return_sequences=False))
    #model.add(Dropout(rate=hp.Float('dropout_2', min_value=0, max_value=0.8, step=0.05)))
    model.add(Dense(1))

    # 设置初始学习率
    initial_learning_rate = 1e-4
    model.compile(optimizer=Adam(learning_rate=initial_learning_rate), loss='mae', metrics=['mse', 'mae'])
    return model


# 超参数搜索和训练函数
def combined_search_tuning(x_train, y_train):
    # 随机搜索配置
    tuner_random = kt.RandomSearch(
        build_model,
        objective='val_loss',
        max_trials=30,
        executions_per_trial=2,
        directory='my_dir',
        project_name='lstm_random_search_'
    )

    # 进行随机搜索
    tuner_random.search(x_train, y_train, epochs=15, validation_split=0.2)
    best_hps_random = tuner_random.get_best_hyperparameters(num_trials=1)[0]

    # 网格搜索配置
    tuner_grid = kt.Hyperband(
        build_model,
        objective='val_loss',
        max_epochs=20,
        factor=2,
        directory='my_dir',
        project_name='lstm_grid_search',
        overwrite=True  # 确保每次搜索都是全新的
    )

    # 设置搜索范围
    tuner_grid.oracle.hyperparameters.Int('units', min_value=best_hps_random.get('units') - 32, max_value=best_hps_random.get('units') + 32, step=4)
    tuner_grid.oracle.hyperparameters.Float('dropout_1', min_value=max(0, best_hps_random.get('dropout_1') - 0.1), max_value=min(1, best_hps_random.get('dropout_1') + 0.1), step=0.05)

    # 进行网格搜索
    tuner_grid.search(x_train, y_train, epochs=15, validation_split=0.2)
    return tuner_grid.get_best_hyperparameters(num_trials=1)[0]

# 获取最优超参数
best_hps_combined = combined_search_tuning(x_train_rainy, y_train_rainy)

# 打印最优超参数
best_hyperparameters = {
    'units': best_hps_combined.get('units'),
    'dropout_1': best_hps_combined.get('dropout_1'),
    #'dropout_2': best_hps_combined.get('dropout_2'),
}
print("最优超参数：", best_hyperparameters)


#188 0.15 1
# 训练模型
model = build_model(best_hps_combined)
history = model.fit(x_train_rainy, y_train_rainy, epochs=100, batch_size=64, validation_split=0.2)

### 网格搜索

In [ ]:
import keras_tuner as kt
from keras.models import Sequential
from keras.layers import LSTM, Dense, Dropout, Masking
from keras.optimizers import Adam

# LSTM模型构建函数
def build_model(hp):
    model = Sequential()
    model.add(Masking(mask_value=0, input_shape=(TIME_STEPS, INPUT_DIMS)))
    model.add(LSTM(units=hp.Int('units', min_value=188 - 32, max_value=188 + 32, step=8),
                   return_sequences=True))
    model.add(Dropout(rate=hp.Float('dropout_1', min_value=0.15 - 0.1, max_value=0.15 + 0.1, step=0.05)))
    model.add(LSTM(units=hp.Int('units_l2', min_value=108 - 32, max_value=108 + 32, step=8),
                   return_sequences=False))
    model.add(Dense(1))

    # 设置初始学习率
    initial_learning_rate = 1e-4
    model.compile(optimizer=Adam(learning_rate=initial_learning_rate), loss='mae', metrics=['mse', 'mae'])
    return model

# 超参数网格搜索函数
def grid_search_tuning(x_train, y_train):
    tuner = kt.Hyperband(
        build_model,
        objective='val_loss',
        max_epochs=20,
        factor=2,
        directory='my_dir',
        project_name='lstm_grid_search',
        overwrite=True  # 确保每次搜索都是全新的
    )

    tuner.search(x_train, y_train, epochs=15, validation_split=0.2)
    return tuner.get_best_hyperparameters(num_trials=1)[0]

# 设置数据的时间步长和输入维度
TIME_STEPS = x_train_rainy.shape[1]
INPUT_DIMS = x_train_rainy.shape[2]

# 获取最优超参数
best_hps = grid_search_tuning(x_train_rainy, y_train_rainy)

# 打印最优超参数
best_hyperparameters = {
    'units': best_hps.get('units'),
    'dropout_1': best_hps.get('dropout_1'),
    'units_l2': best_hps.get('units_l2')
}
print("最优超参数：", best_hyperparameters)

# 使用最优超参数训练模型 {'units': 204, 'dropout_1': 0.25, 'units_l2': 116}
model = build_model(best_hps)
history = model.fit(x_train_rainy, y_train_rainy, epochs=30, batch_size=64, validation_split=0.2)

## <font face="宋体" size=5 color=#A52A2A>全局数据训练和评估
- <font face="宋体" size=3 color=#A52A2A>处理策略：
    1. for 循环，对每一个子区域循环
    2. 循环：
        - 子区域数据划分(先定义边界大小，现可通过双重循环得到数据集)
        - 缺失数据mask和mask矩阵获取（现可直接得到）
        - 子区域模型训练，同时训练rainy和dry模型
        - 子区域数据评估，rainy和dry分别评估，分开之后再合并评估（评估还得分train和test评估）
        - filling数据如何合并到原始数据中去？（还需要indx标签）评估之后估计filling数据最后生成数据集
            1. 是否根本不需要这种模型？而是直接对所有数据划分后全都进行预测生成最后数据集？
            2. 而之前的部分仅用于评估模型如何？！
        - 需保存y_inv_grid训练结束仅保存模型，模型名以lon=\*\*,lat=\*\*,rainy/dry区分
    3. 返回mae/mse/r2_grid矩阵，以及y_inv_grid矩阵，后面绘制两种图像，分别是区域内粗和细分辨率误图像

### <font face="宋体" size=4 color=#A52A2A>函数准备

In [1]:
import datetime
from sklearn.preprocessing import MinMaxScaler
import numpy as np
import numpy as np
import pandas as pd
import xarray as xr
import keras_tuner as kt
from keras.models import Sequential
from keras.layers import LSTM, Dense, Dropout, Masking
from keras.optimizers import Adam
from keras.callbacks import LearningRateScheduler, LambdaCallback
import keras.backend as K
import keras.backend as K
from keras.layers import Bidirectional,LSTM, Dense, Dropout
from keras.models import Sequential, Model
from keras.layers import Dense, LSTM,Flatten,BatchNormalization
from keras.layers.core import Dense, Dropout, Activation, Flatten
from keras.utils import np_utils
from keras.layers import Masking, Input
from keras.callbacks import LearningRateScheduler, EarlyStopping
from keras.optimizers import Adam
from keras.utils import plot_model
from keras.models import load_model
from keras.layers import Masking, Input
from sklearn.preprocessing import StandardScaler
from keras.callbacks import LearningRateScheduler
from keras.callbacks import EarlyStopping
import numpy as np
import tensorflow as tf
import keras_tuner as kt
import keras.backend as K
from keras.layers import Bidirectional,LSTM, Dense, Dropout
from keras.models import Sequential, Model
from keras.layers import Dense, LSTM,Flatten,BatchNormalization
from keras.layers.core import Dense, Dropout, Activation, Flatten
from keras.utils import np_utils
from keras.layers import Masking, Input
from keras.callbacks import LearningRateScheduler, EarlyStopping
from keras.optimizers import Adam
from keras.utils import plot_model
from keras.models import load_model
from keras.layers import Masking, Input
from sklearn.preprocessing import StandardScaler
from keras.callbacks import LearningRateScheduler
from keras.callbacks import EarlyStopping
import numpy as np
import tensorflow as tf
from sklearn.metrics import r2_score, mean_squared_error, mean_absolute_error
import os
from sklearn.model_selection import train_test_split
import os
import yaml
import sys
import math
#sys.path.append('C:\\Users\\Administrator\\Desktop\\code\\ST-Conv')
import argparse
import pickle
import xarray as xr
import numpy as np
import pandas as pd 
import random
from datetime import datetime, timedelta
from utils.utils import Util

In [2]:
def load_config(file_path):
    with open(file_path, 'r', encoding='utf-8') as f:
        config = yaml.load(f, Loader=yaml.FullLoader)
    return config
# 数据预准备函数
def data_preparation(file_path, target_var, var_list, missing_values):
    # 读取nc文件
    data = xr.open_dataset(file_path)

    # 将lat坐标轴从小到大排列并选择特定区域
    data = data.sortby('lat')

    # 提取起始时间和结束时间并转化为datatime对象
    end_time = pd.to_datetime(data.time.values[-1])

    # 创建三个新的变量：Longitude, Latitude, DOY
    longitude = xr.DataArray(np.zeros(data[target_var].shape), 
                            dims=data[target_var].dims, 
                            coords=data[target_var].coords)
    latitude = xr.DataArray(np.zeros(data[target_var].shape), 
                            dims=data[target_var].dims, 
                            coords=data[target_var].coords)
    doy = xr.DataArray(np.zeros(data[target_var].shape), 
                    dims=data[target_var].dims, 
                    coords=data[target_var].coords)

    # 更新 DOY 变量
    for i in range(data.time.shape[0]):
        timestamp = pd.Timestamp(data.time[i].values)
        doy[i, :, :] = Util.calculate_hours_since_year_start(timestamp)

    # 更新 Longitude 和 Latitude 变量
    for i in range(data.lon.shape[0]):
        longitude[:, :, i] = data.lon[i]
    for i in range(data.lat.shape[0]):
        latitude[:, i, :] = data.lat[i]

    # 将新的变量添加到数据集中
    data = data.assign(Longitude=longitude, Latitude=latitude, DOY=doy)

    # 只提取var_list中的变量，如果var_list中包含新添加的变量，则它们也会被包括
    data = data[var_list]

    # 质量控制
    # MYD13A1:有效值-2000 to 10000，其他设置为0
    data.NDVI.values = np.where((data.NDVI.values < -2000) | (data.NDVI.values > 10000), 0, data.NDVI.values)
    data.EVI.values = np.where((data.EVI.values < -2000) | (data.EVI.values > 10000), 0, data.EVI.values)
    # MCD11A1:有效值7500 to 65535 
    data.LST_Day_1km.values = np.where((data.LST_Day_1km.values < 7500) | (data.LST_Day_1km.values > 65535), 0, data.LST_Day_1km.values)
    data.LST_Night_1km.values = np.where((data.LST_Night_1km.values < 7500) | (data.LST_Night_1km.values > 65535), 0, data.LST_Night_1km.values)
    # MCD43C3：有效值 0 到 32766
    MCD43C3_Var = ['Albedo_BSA_Band1','Albedo_BSA_Band2','Albedo_BSA_Band3','Albedo_BSA_Band4','Albedo_BSA_Band5','Albedo_BSA_Band6','Albedo_BSA_Band7','Albedo_BSA_vis','Albedo_BSA_nir','Albedo_BSA_shortwave'\
                   ,'Albedo_WSA_vis','Albedo_WSA_nir','Albedo_WSA_shortwave','Albedo_WSA_Band1','Albedo_WSA_Band2','Albedo_WSA_Band3','Albedo_WSA_Band4','Albedo_WSA_Band5','Albedo_WSA_Band6','Albedo_WSA_Band7']
    #循环遍历所有变量      
    for var in MCD43C3_Var:
        data[var].values = np.where((data[var].values < 0) | (data[var].values > 32766), 0, data[var].values)
    # 使用 missing_values 字典辨别缺失值
    for var, missing_val in missing_values.items():
        if var in data:
            data[var] = data[var].where(data[var] != missing_val, np.nan)
    data = data.isel(time=slice(0,-1))
    return data

# 标准化和逆标准化
def MinMaxNormalization(data):
    """
    为时间序列数据进行Min-Max标准化的函数，标准化时自动排除列的缺失值。
    :param data: 数据的形状(time,lon,lat,var)
    :return: 标准化后的数据和标准化参数矩阵。
    """
    data = np.array(data)

    # 计算最小值和最大值
    min_vals = np.nanmin(data, axis=0)
    max_vals = np.nanmax(data, axis=0)
    
    # 防止分母为0
    range_vals = max_vals - min_vals
    range_vals[range_vals == 0] = 1e-10
    
    # 创建标准化参数矩阵
    normalize_mat = np.stack((min_vals, range_vals), axis=-1)
    normalized_data = (data - min_vals) / range_vals

    return normalized_data, normalize_mat
def IMinMaxNormalization(data, normalize_matrix):
    """
    为Min-Max标准化的数据进行逆标准化的函数
    :param data: 数据的形状应和mat形状一致为：(**time,*lon,*lat,var，*2)
    :param normalize_matrix: 标准化参数矩阵
    :return: 逆标准化后的数据
    """
    data = np.array(data)

    min_vals = normalize_matrix[..., 0]
    range_vals = normalize_matrix[..., 1]
    
    inv_normalized_data = (data * range_vals) + min_vals
    
    return inv_normalized_data

# 区间数据调用函数，输入经纬度区域序号从1开始,判断是否lon_max和lat_max是否大于序列最大值（取序列最大值）
def select_data(data,lon_num,lat_num,area_size):
    '''
    data:输入数据
    lon_num:经度区域序号
    lat_num:纬度区域序号
    area_size:区域尺寸
    '''
    lon_shape = data.lon.shape[0]
    lat_shape = data.lat.shape[0]
    # 如果最小值大于序列最大值，报错
    if lon_num>lon_shape//area_size or lat_num>lat_shape//area_size:
        print("Error:lon_num or lat_num is too large")
    lon_min = data.lon[(lon_num-1)*area_size]
    lat_min = data.lat[(lat_num-1)*area_size]
    if lon_num*area_size>lon_shape:
        lon_max = data.lon[lon_shape-1]
    elif lon_num*area_size<=lon_shape:
        lon_max = data.lon[lon_num*area_size-1]
    if lat_num*area_size>lat_shape:
        lat_max = data.lat[lat_shape-1]
    elif lat_num*area_size<=lat_shape:
        lat_max = data.lat[lat_num*area_size-1]
    data_subset = data.sel(lon=slice(lon_min, lon_max), lat=slice(lat_min, lat_max))
    return data_subset

# 掩码矩阵函数
def mask_fun(area_data,missing_values,var_sets):
    area_dataset = area_data.to_array().values
    # 依据 var_sets 顺序创建一个掩码矩阵数组
    mask_array = np.array([np.where(area_data[var] == missing_values[var], 1, 0) for var in var_sets])
    if mask_array.shape == area_dataset.shape:
        masked_area_dataset = np.where(mask_array, np.nan, area_dataset)
    else:
        raise ValueError("掩码矩阵与数据矩阵形状不匹配")

    #维度变化，将变量的维度从最开始的维度转换为第四个维度
    masked_area_dataset,mask_array = np.transpose(masked_area_dataset, (1, 2, 3, 0)),np.transpose(mask_array, (1, 2, 3, 0))
    return masked_area_dataset,mask_array

# 雨季旱季划分函数(5,6,7,8,9,10月雨季，11,12,1,2,3,4旱季)
def split_rainy_dry_data_to_filling(data_point, TIME_STEPS):
    '''
    直接将传入的点数据划分为雨季旱季，雨季旱季数据分别填充,返回了两个序列标识，对应两个数组的拼接序号
    '''
    x_rainy,x_dry, x_rainy_ind,x_dry_ind = [],[],[],[]
    #  x_rainy_ind,x_dry_ind记录各自序号,从0开始
    
    for i in range(data_point.shape[0] - TIME_STEPS):
        current_date = datetime.datetime(2016, 1, 1) + datetime.timedelta(days=i//2)
        X_sample = data_point[i:i+TIME_STEPS]
        # 判断雨季旱季
        if current_date.month in [5, 6, 7, 8, 9, 10]:
            x_rainy.append(X_sample)
            x_rainy_ind = np.append(x_rainy_ind,i)
        else:
            x_dry.append(X_sample)
            x_dry_ind = np.append(x_dry_ind,i)
    x_rainy,x_dry ,x_rainy_ind,x_dry_ind= np.array(x_rainy),np.array(x_dry),np.array(x_rainy_ind),np.array(x_dry_ind)
    return x_rainy,x_dry,x_rainy_ind,x_dry_ind
def area_split_data_to_filling(data,TIME_STEPS):
    '''
    传入区域数据，逐个划分，返回三维grid
    data为narray
    '''
    lon,lat = data.shape[1],data.shape[2]
    x_rainy_filling_grid,x_dry_filling_grid,x_rainy_ind_grid,x_dry_ind_grid = np.empty((lon,lat), dtype=object),np.empty((lon,lat), dtype=object),np.empty((lon,lat), dtype=object),np.empty((lon,lat), dtype=object)
    for i in range(lon):
        for j in range(lat):
            point_data = data[:,i,j,:]
            x_rainy_filling_grid_temp,x_dry_filling_grid_temp,x_rainy_ind_grid_temp,x_dry_ind_grid_temp = split_rainy_dry_data_to_filling(point_data, TIME_STEPS)
            x_rainy_filling_grid[i,j],x_dry_filling_grid[i,j],x_rainy_ind_grid[i,j],x_dry_ind_grid[i,j] = np.nan_to_num(x_rainy_filling_grid_temp),np.nan_to_num(x_dry_filling_grid_temp),x_rainy_ind_grid_temp,x_dry_ind_grid_temp
            
    return x_rainy_filling_grid,x_dry_filling_grid,x_rainy_ind_grid,x_dry_ind_grid
def split_rainy_dry_data(data_point, normal_mat, normal_methods, timestamps, TIME_STEPS):
    """
    单点雨旱季划分函数，数据全填充即为规则数据
    :param data_point: 单点数据
    :param normal_mat: 归一化矩阵
    :param normal_methods: 归一化方法
    :param TIME_STEPS: 时间步长
    :return: 划分后的数据
    """
    x_rainy, y_rainy, x_test_rainy, y_test_rainy, x_dry, y_dry, x_test_dry, y_test_dry, x_filling_rainy, x_filling_dry = [], [], [], [], [], [], [], [], [], []
    train_end_date = datetime.datetime(2023, 1, 1)  # 训练集结束时间
    for i in range(data_point.shape[0] - TIME_STEPS):
        current_date = datetime.datetime(2015, 3, 31) + datetime.timedelta(days=i//2)
        X_sample = data_point[i:i+TIME_STEPS]
        y_sample = data_point[i+TIME_STEPS, 0]
        # 判断雨季旱季
        if current_date.month in [5, 6, 7, 8, 9, 10]:
            if np.isnan(y_sample):
                x_filling_rainy.append(X_sample)
            elif current_date < train_end_date:
                x_rainy.append(X_sample)
                y_rainy.append(y_sample)
            else:
                x_test_rainy.append(X_sample)
                y_test_rainy.append(y_sample)
        else:
            if np.isnan(y_sample):
                x_filling_dry.append(X_sample)
            elif current_date < train_end_date:
                x_dry.append(X_sample)
                y_dry.append(y_sample)
            else:
                x_test_dry.append(X_sample)
                y_test_dry.append(y_sample)

    # 反归一化处理
    x_rainy, y_rainy = np.array(x_rainy), np.array(y_rainy)
    x_dry, y_dry = np.array(x_dry), np.array(y_dry)
    x_test_rainy, y_test_rainy = np.array(x_test_rainy), np.array(y_test_rainy)
    x_test_dry, y_test_dry = np.array(x_test_dry), np.array(y_test_dry)
    x_filling_rainy = np.array(x_filling_rainy)
    x_filling_dry = np.array(x_filling_dry)
    if normal_methods == "MinMax":
        y_true_train_rainy = IMinMaxNormalization(y_rainy, normal_mat[0, :])
        y_true_test_rainy = IMinMaxNormalization(y_test_rainy, normal_mat[0, :])
        y_true_train_dry = IMinMaxNormalization(y_dry, normal_mat[0, :])
        y_true_test_dry = IMinMaxNormalization(y_test_dry, normal_mat[0, :])
    else:
        y_true_train_rainy = INormalization(y_rainy, normal_mat[0, :])
        y_true_test_rainy = INormalization(y_test_rainy, normal_mat[0, :])
        y_true_train_dry = INormalization(y_dry, normal_mat[0, :])
        y_true_test_dry = INormalization(y_test_dry, normal_mat[0, :])

    return x_rainy, y_rainy, y_true_train_rainy, x_test_rainy, y_test_rainy, y_true_test_rainy, x_filling_rainy, x_dry, y_dry, y_true_train_dry, x_test_dry, y_test_dry, y_true_test_dry, x_filling_dry
def split_rainy_dry_data_without_filling(data_point, normal_mat, normal_methods, timestamps, TIME_STEPS):
    """
    单点雨旱季划分函数，数据全填充即为规则数据
    :param data_point: 单点数据
    :param normal_mat: 归一化矩阵
    :param normal_methods: 归一化方法
    :param TIME_STEPS: 时间步长
    :return: 划分后的数据
    """
    x_rainy, y_rainy, x_test_rainy, y_test_rainy, x_dry, y_dry, x_test_dry, y_test_dry = [], [], [], [], [], [], [], []
    train_end_date = datetime.datetime(2023, 1, 1)  # 训练集结束时间
    for i in range(data_point.shape[0] - TIME_STEPS):
        current_date = datetime.datetime(2015, 3, 31) + datetime.timedelta(days=i//2)
        X_sample = data_point[i:i+TIME_STEPS]
        y_sample = data_point[i+TIME_STEPS, 0]
        # 判断雨季旱季
        if current_date.month in [5, 6, 7, 8, 9, 10]:
            if np.isnan(y_sample):
                continue
            elif current_date < train_end_date:
                x_rainy.append(X_sample)
                y_rainy.append(y_sample)
            else:
                x_test_rainy.append(X_sample)
                y_test_rainy.append(y_sample)
        else:
            if np.isnan(y_sample):
                continue
            elif current_date < train_end_date:
                x_dry.append(X_sample)
                y_dry.append(y_sample)
            else:
                x_test_dry.append(X_sample)
                y_test_dry.append(y_sample)

    # 反归一化处理
    x_rainy, y_rainy = np.array(x_rainy), np.array(y_rainy)
    x_dry, y_dry = np.array(x_dry), np.array(y_dry)
    x_test_rainy, y_test_rainy = np.array(x_test_rainy), np.array(y_test_rainy)
    x_test_dry, y_test_dry = np.array(x_test_dry), np.array(y_test_dry)
    if normal_methods == "MinMax":
        y_true_train_rainy = IMinMaxNormalization(y_rainy, normal_mat[0, :])
        y_true_test_rainy = IMinMaxNormalization(y_test_rainy, normal_mat[0, :])
        y_true_train_dry = IMinMaxNormalization(y_dry, normal_mat[0, :])
        y_true_test_dry = IMinMaxNormalization(y_test_dry, normal_mat[0, :])
    else:
        y_true_train_rainy = INormalization(y_rainy, normal_mat[0, :])
        y_true_test_rainy = INormalization(y_test_rainy, normal_mat[0, :])
        y_true_train_dry = INormalization(y_dry, normal_mat[0, :])
        y_true_test_dry = INormalization(y_test_dry, normal_mat[0, :])

    return x_rainy, y_rainy, y_true_train_rainy, x_test_rainy, y_test_rainy, y_true_test_rainy,  x_dry, y_dry, y_true_train_dry, x_test_dry, y_test_dry, y_true_test_dry
def split_rainy_dry_data_random_without_filling(data_point, normal_mat, normal_methods, timestamps, TIME_STEPS):
    """
    单点雨旱季划分函数，数据划分了旱季雨季之后，随机划分，不以时间节点划分测试集验证集，数据全填充即为规则数据
    :param data_point: 单点数据
    :param normal_mat: 归一化矩阵
    :param normal_methods: 归一化方法
    :param TIME_STEPS: 时间步长
    :return: 划分后的数据
    """
    x_rainy, y_rainy, x_test_rainy, y_test_rainy, x_dry, y_dry, x_test_dry, y_test_dry = [], [], [], [], [], [], [], []
    for i in range(data_point.shape[0] - TIME_STEPS):
        current_date = datetime.datetime(2016, 1, 1) + datetime.timedelta(days=i//2)
        X_sample = data_point[i:i+TIME_STEPS]
        y_sample = data_point[i+TIME_STEPS, 0]
        # 判断雨季旱季
        if current_date.month in [5, 6, 7, 8, 9, 10]:
            if np.isnan(y_sample):
                continue
            else:
                x_rainy.append(X_sample)
                y_rainy.append(y_sample)
        else:
            if np.isnan(y_sample):
                continue
            else:
                x_dry.append(X_sample)
                y_dry.append(y_sample)
    # 80%训练集，20%测试集;训练集\验证集训练时候再划分
    if len(x_rainy) and len(x_dry) > 20:
        x_train_rainy, x_test_rainy, y_train_rainy, y_test_rainy = train_test_split(x_rainy, y_rainy, test_size=0.2, random_state=42)
        x_train_dry, x_test_dry, y_train_dry, y_test_dry = train_test_split(x_dry, y_dry, test_size=0.2, random_state=42)
    else :
        x_train_rainy, y_train_rainy = x_rainy, y_rainy
        x_train_dry, y_train_dry = x_dry, y_dry
        x_test_rainy, y_test_rainy = [], []
        x_test_dry, y_test_dry = [], []
    # 反归一化处理
    x_train_rainy, y_train_rainy = np.array(x_rainy), np.array(y_rainy)
    x_train_dry, y_train_dry = np.array(x_dry), np.array(y_dry)
    x_test_rainy, y_test_rainy = np.array(x_test_rainy), np.array(y_test_rainy)
    x_test_dry, y_test_dry = np.array(x_test_dry), np.array(y_test_dry)
    if normal_methods == "MinMax":
        y_true_train_rainy = IMinMaxNormalization(y_rainy, normal_mat[0, :])
        y_true_test_rainy = IMinMaxNormalization(y_test_rainy, normal_mat[0, :])
        y_true_train_dry = IMinMaxNormalization(y_dry, normal_mat[0, :])
        y_true_test_dry = IMinMaxNormalization(y_test_dry, normal_mat[0, :])
    else:
        y_true_train_rainy = INormalization(y_rainy, normal_mat[0, :])
        y_true_test_rainy = INormalization(y_test_rainy, normal_mat[0, :])
        y_true_train_dry = INormalization(y_dry, normal_mat[0, :])
        y_true_test_dry = INormalization(y_test_dry, normal_mat[0, :])

    return x_train_rainy, y_train_rainy, y_true_train_rainy, x_test_rainy, y_test_rainy, y_true_test_rainy,  x_train_dry, y_train_dry, y_true_train_dry, x_test_dry, y_test_dry, y_true_test_dry

def split_data_random_without_filling(data_point, normal_mat, normal_methods, TIME_STEPS):
    """
    单点数据随机划分，不以时间节点划分测试集验证集，数据全填充即为规则数据
    :param data_point: 单点数据
    :param normal_mat: 归一化矩阵
    :param normal_methods: 归一化方法
    :param TIME_STEPS: 时间步长
    :return: 划分后的数据
    """
    x, y = [], []
    for i in range(data_point.shape[0] - TIME_STEPS):
        X_sample = data_point[i:i+TIME_STEPS]
        y_sample = data_point[i+TIME_STEPS, 0]
        if np.isnan(y_sample):
            continue
        else:
            x.append(X_sample)
            y.append(y_sample)

    # 80%训练集，20%测试集;训练集\验证集训练时候再划分
    if len(x) > 20:
        x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.2, random_state=42)
    else:
        x_train, y_train = x, y
        x_test, y_test = [], []

    # 反归一化处理
    x_train, y_train = np.array(x_train), np.array(y_train)
    x_test, y_test = np.array(x_test), np.array(y_test)
    if normal_methods == "MinMax":
        y_true_train = IMinMaxNormalization(y_train, normal_mat[0, :])
        y_true_test = IMinMaxNormalization(y_test, normal_mat[0, :])
    else:
        y_true_train = INormalization(y_train, normal_mat[0, :])
        y_true_test = INormalization(y_test, normal_mat[0, :])

    return x_train, y_train, y_true_train, x_test, y_test, y_true_test
# 全域数据划分函数（只完善了需要使用的部分）
def area_split_data_without_filling(area_data, area_size, MinMaxNormalization_data, MinMaxNormalization_matrix, timestamps, methods_first="rainydry", methods_second="MinMax", time_step= 60):
    if methods_first == "rainydry":
        # 初始化地理网格的x数据集,每个的形状为：area_data.lon.shape[0],area_data.lat.shape[0]，~
        lon_size,lat_size = area_size,area_size
        x_train_rainy_grid, x_test_rainy_grid, x_train_dry_grid, x_test_dry_grid = np.empty((lon_size, lat_size), dtype=object),np.empty((lon_size, lat_size), dtype=object),np.empty((lon_size, lat_size), dtype=object),np.empty((lon_size, lat_size), dtype=object)
        y_true_train_rainy_grid, y_true_test_rainy_grid, y_true_train_dry_grid, y_true_test_dry_grid = np.empty((lon_size, lat_size), dtype=object), np.empty((lon_size, lat_size), dtype=object),np.empty((lon_size, lat_size), dtype=object),np.empty((lon_size, lat_size), dtype=object)
        
        # 初始化训练集和测试集
        x_train_rainy, y_train_rainy, y_train_true_rainy,  x_test_rainy, y_test_rainy, y_test_true_rainy,\
            x_train_dry, y_train_dry, y_train_true_dry,  x_test_dry, y_test_dry, y_test_true_dry,  = [], [], [], [], [], [],[], [], [],[], [], []
        
        var_list = [x_train_rainy, y_train_rainy, y_train_true_rainy,  x_test_rainy, y_test_rainy, y_test_true_rainy,\
                     x_train_dry, y_train_dry, y_train_true_dry,  x_test_dry, y_test_dry, y_test_true_dry]
        # 数据(MinMaxNormalization_data)划分为样本
        for i in range(area_data.lon.shape[0]):
            for j in range(area_data.lat.shape[0]):
                #如果目标变量在这个点的所有时间点上全为nan，就跳过这个点
                if area_data.isel(lon=i, lat=j)[target_var].isnull().all():
                    continue
                else:
                    point_data = MinMaxNormalization_data[:,i, j, :]
                    x_train_point_rainy, y_train_point_rainy, y_true_train_point_rainy, x_test_point_rainy, y_test_point_rainy, y_true_test_point_rainy, \
                        x_train_point_dry, y_train_point_dry, y_true_train_point_dry,  \
                            x_test_point_dry, y_test_point_dry, y_true_test_point_dry = \
                                    split_rainy_dry_data_random_without_filling(point_data, MinMaxNormalization_matrix[i,j,:], methods_second, timestamps, time_step)
                    
                    # 地理网格赋值
                    x_train_rainy_grid[i,j],x_test_rainy_grid[i,j], x_train_dry_grid[i,j], x_test_dry_grid[i,j] = \
                        np.nan_to_num(x_train_point_rainy), np.nan_to_num(x_test_point_rainy), np.nan_to_num(x_train_point_dry), np.nan_to_num(x_test_point_dry)
                    y_true_train_rainy_grid[i,j], y_true_test_rainy_grid[i,j], y_true_train_dry_grid[i,j], y_true_test_dry_grid[i,j] = \
                        y_true_train_point_rainy, y_true_test_point_rainy, y_true_train_point_dry, y_true_test_point_dry

                    var_list_point = [x_train_point_rainy, y_train_point_rainy, y_true_train_point_rainy,  x_test_point_rainy, y_test_point_rainy, \
                    y_true_test_point_rainy, x_train_point_dry, y_train_point_dry, y_true_train_point_dry, \
                    x_test_point_dry, y_test_point_dry, y_true_test_point_dry]
                    
                    # 判断输出的每一个数据，只有形状为(0,)的数据才append,为(0,)的数据跳过
                    # 使用for循环判断每一个数据是否为(0,)，如果是(0,)就跳过，不是(0,)就append
                    for var, var_point in zip(var_list, var_list_point):
                        if var_point.shape != (0,):
                            var.append(var_point)
        if x_test_rainy:
            '''
            # 使用循环判断
            for var in var_list:
                var = np.concatenate(var, axis=0)
            '''
            x_train_rainy = np.concatenate(x_train_rainy, axis=0)
            y_train_rainy = np.concatenate(y_train_rainy, axis=0)
            y_train_true_rainy = np.concatenate(y_train_true_rainy, axis=0)
            x_test_rainy = np.concatenate(x_test_rainy, axis=0)
            y_test_rainy = np.concatenate(y_test_rainy, axis=0)
            y_test_true_rainy = np.concatenate(y_test_true_rainy, axis=0)
            x_train_dry = np.concatenate(x_train_dry, axis=0)
            y_train_dry = np.concatenate(y_train_dry, axis=0)
            y_train_true_dry = np.concatenate(y_train_true_dry, axis=0)
            x_test_dry = np.concatenate(x_test_dry, axis=0)
            y_test_dry = np.concatenate(y_test_dry, axis=0)
            y_test_true_dry = np.concatenate(y_test_true_dry, axis=0)
        # 将x_train/x_test/x_filling的所有nan值替换为0
        x_train_rainy = np.nan_to_num(x_train_rainy)
        x_test_rainy = np.nan_to_num(x_test_rainy)
        x_train_dry = np.nan_to_num(x_train_dry)
        x_test_dry = np.nan_to_num(x_test_dry)
        return x_train_rainy, y_train_rainy, y_train_true_rainy, x_test_rainy, y_test_rainy, y_test_true_rainy, \
             x_train_dry, y_train_dry, y_train_true_dry, x_test_dry, y_test_dry, y_test_true_dry, \
                 x_train_rainy_grid, x_test_rainy_grid, x_train_dry_grid, x_test_dry_grid,\
                    y_true_train_rainy_grid, y_true_test_rainy_grid, y_true_train_dry_grid, y_true_test_dry_grid
    else:
        # 初始化训练集和测试集
        x_train, y_train, y_train_true, x_test, y_test, y_test_true = [], [], [], [], [], []
        var_list = [x_train, y_train, y_train_true, x_test, y_test, y_test_true]

        # 数据(MinMaxNormalization_data)划分为样本
        for i in range(area_data.lon.shape[0]):
            for j in range(area_data.lat.shape[0]):
                #如果目标变量在这个点的所有时间点上全为nan，就跳过这个点
                if area_data.isel(lon=i, lat=j)[target_var].isnull().all():
                    continue
                else:
                    point_data = MinMaxNormalization_data[:,i, j, :]
                    x_train_point, y_train_point, y_true_train_point,  x_test_point, y_test_point, y_true_test_point = split_regular_data(point_data,  MinMaxNormalization_matrix[i,j,:], methods_second, time_step)
                    var_list_point = [x_train_point, y_train_point, y_true_train_point, x_test_point, y_test_point, y_true_test_point]
                    for var, var_point in zip(var_list, var_list_point):
                        if var_point.shape != (0,):
                            var.append(var_point)
        if x_test:
            x_train = np.concatenate(x_train, axis=0)
            y_train = np.concatenate(y_train, axis=0)
            y_train_true = np.concatenate(y_train_true, axis=0)
            x_test = np.concatenate(x_test, axis=0)
            y_test = np.concatenate(y_test, axis=0)
            y_test_true = np.concatenate(y_test_true, axis=0)

        # 将x_train/x_test/x_filling的所有nan值替换为0
        x_train = np.nan_to_num(x_train)
        x_test = np.nan_to_num(x_test)

        return x_train, y_train, y_train_true, x_test, y_test, y_test_true
def area_split_data(area_data, MinMaxNormalization_data, MinMaxNormalization_matrix, timestamps, methods_first="rainydry", methods_second="MinMax", time_step= 60):
    if methods_first == "rainydry":
        # 初始化地理网格的x数据集,每个的形状为：area_data.lon.shape[0],area_data.lat.shape[0]，~
        lon_size,lat_size = area_data.lon.shape[0],area_data.lat.shape[0]
        x_train_rainy_grid, x_test_rainy_grid, x_filling_rainy_grid, x_train_dry_grid, x_test_dry_grid, x_filling_dry_grid = np.empty((lon_size, lat_size), dtype=object), np.empty((lon_size, lat_size), dtype=object),np.empty((lon_size, lat_size), dtype=object),np.empty((lon_size, lat_size), dtype=object),np.empty((lon_size, lat_size), dtype=object),np.empty((lon_size, lat_size), dtype=object)
        y_true_train_rainy_grid, y_true_test_rainy_grid, y_true_train_dry_grid, y_true_test_dry_grid = np.empty((lon_size, lat_size), dtype=object), np.empty((lon_size, lat_size), dtype=object),np.empty((lon_size, lat_size), dtype=object),np.empty((lon_size, lat_size), dtype=object)
        
        # 初始化训练集和测试集
        x_train_rainy, y_train_rainy, y_train_true_rainy,  x_test_rainy, y_test_rainy, y_test_true_rainy,  x_filling_rainy,  \
            x_train_dry, y_train_dry, y_train_true_dry,  x_test_dry, y_test_dry, y_test_true_dry,  x_filling_dry,  = [], [], [], [], [], [],[], [], [],[], [], [],[], []
        
        var_list = [x_train_rainy, y_train_rainy, y_train_true_rainy,  x_test_rainy, y_test_rainy, y_test_true_rainy,  x_filling_rainy, \
                     x_train_dry, y_train_dry, y_train_true_dry,  x_test_dry, y_test_dry, y_test_true_dry, x_filling_dry]
        # 数据(MinMaxNormalization_data)划分为样本
        for i in range(area_data.lon.shape[0]):
            for j in range(area_data.lat.shape[0]):
                #如果目标变量在这个点的所有时间点上全为nan，就跳过这个点
                if area_data.isel(lon=i, lat=j)[target_var].isnull().all():
                    continue
                else:
                    point_data = MinMaxNormalization_data[:,i, j, :]
                    x_train_point_rainy, y_train_point_rainy, y_true_train_point_rainy, x_test_point_rainy, y_test_point_rainy, y_true_test_point_rainy, \
                         x_filling_point_rainy,x_train_point_dry, y_train_point_dry, y_true_train_point_dry,  \
                            x_test_point_dry, y_test_point_dry, y_true_test_point_dry, x_filling_point_dry = \
                                    split_rainy_dry_data(point_data, MinMaxNormalization_matrix[i,j,:], methods_second, timestamps, time_step)
                    
                    # 地理网格赋值
                    x_train_rainy_grid[i,j],x_test_rainy_grid[i,j], x_filling_rainy_grid[i,j], x_train_dry_grid[i,j], x_test_dry_grid[i,j], x_filling_dry_grid[i,j] = \
                        np.nan_to_num(x_train_point_rainy), np.nan_to_num(x_test_point_rainy), np.nan_to_num(x_filling_point_rainy), np.nan_to_num(x_train_point_dry), np.nan_to_num(x_test_point_dry), np.nan_to_num(x_filling_point_dry)
                    y_true_train_rainy_grid[i,j], y_true_test_rainy_grid[i,j], y_true_train_dry_grid[i,j], y_true_test_dry_grid[i,j] = \
                        y_true_train_point_rainy, y_true_test_point_rainy, y_true_train_point_dry, y_true_test_point_dry

                    var_list_point = [x_train_point_rainy, y_train_point_rainy, y_true_train_point_rainy,  x_test_point_rainy, y_test_point_rainy, \
                    y_true_test_point_rainy,  x_filling_point_rainy,  x_train_point_dry, y_train_point_dry, y_true_train_point_dry, \
                    x_test_point_dry, y_test_point_dry, y_true_test_point_dry,  x_filling_point_dry]
                    
                    # 判断输出的每一个数据，只有形状为(0,)的数据才append,为(0,)的数据跳过
                    # 使用for循环判断每一个数据是否为(0,)，如果是(0,)就跳过，不是(0,)就append
                    for var, var_point in zip(var_list, var_list_point):
                        if var_point.shape != (0,):
                            var.append(var_point)
        if x_test_rainy:
            '''
            # 使用循环判断
            for var in var_list:
                var = np.concatenate(var, axis=0)
            '''
            x_train_rainy = np.concatenate(x_train_rainy, axis=0)
            y_train_rainy = np.concatenate(y_train_rainy, axis=0)
            y_train_true_rainy = np.concatenate(y_train_true_rainy, axis=0)
            x_test_rainy = np.concatenate(x_test_rainy, axis=0)
            y_test_rainy = np.concatenate(y_test_rainy, axis=0)
            y_test_true_rainy = np.concatenate(y_test_true_rainy, axis=0)
            x_filling_rainy = np.concatenate(x_filling_rainy, axis=0)
            x_train_dry = np.concatenate(x_train_dry, axis=0)
            y_train_dry = np.concatenate(y_train_dry, axis=0)
            y_train_true_dry = np.concatenate(y_train_true_dry, axis=0)
            x_test_dry = np.concatenate(x_test_dry, axis=0)
            y_test_dry = np.concatenate(y_test_dry, axis=0)
            y_test_true_dry = np.concatenate(y_test_true_dry, axis=0)
            x_filling_dry = np.concatenate(x_filling_dry, axis=0)
        # 将x_train/x_test/x_filling的所有nan值替换为0
        x_train_rainy = np.nan_to_num(x_train_rainy)
        x_test_rainy = np.nan_to_num(x_test_rainy)
        x_filling_rainy = np.nan_to_num(x_filling_rainy)
        x_train_dry = np.nan_to_num(x_train_dry)
        x_test_dry = np.nan_to_num(x_test_dry)
        x_filling_dry = np.nan_to_num(x_filling_dry)
        return x_train_rainy, y_train_rainy, y_train_true_rainy, x_test_rainy, y_test_rainy, y_test_true_rainy, x_filling_rainy, \
             x_train_dry, y_train_dry, y_train_true_dry, x_test_dry, y_test_dry, y_test_true_dry, x_filling_dry, \
                 x_train_rainy_grid, x_test_rainy_grid, x_filling_rainy_grid, x_train_dry_grid, x_test_dry_grid, x_filling_dry_grid,\
                    y_true_train_rainy_grid, y_true_test_rainy_grid, y_true_train_dry_grid, y_true_test_dry_grid
    else:
        # 初始化训练集和测试集
        x_train, y_train, y_train_true, x_test, y_test, y_test_true, x_filling, = [], [], [], [], [], [],[]
        var_list = [x_train, y_train, y_train_true, x_test, y_test, y_test_true, x_filling]

        # 数据(MinMaxNormalization_data)划分为样本
        for i in range(area_data.lon.shape[0]):
            for j in range(area_data.lat.shape[0]):
                #如果目标变量在这个点的所有时间点上全为nan，就跳过这个点
                if area_data.isel(lon=i, lat=j)[target_var].isnull().all():
                    continue
                else:
                    point_data = MinMaxNormalization_data[:,i, j, :]
                    x_train_point, y_train_point, y_true_train_point,  x_test_point, y_test_point, y_true_test_point,  x_filling_point, t_train_rainy_point, t_test_rainy_point, t_train_dry_point, t_test_dry_point,t_filling_rainy_point, t_filling_dry_point = split_regular_data(point_data,  MinMaxNormalization_matrix[i,j,:], methods_second, time_step)
                    var_list_point = [x_train_point, y_train_point, y_true_train_point, x_test_point, y_test_point, y_true_test_point, x_filling_point,t_train_rainy_point, t_test_rainy_point, t_train_dry_point, t_test_dry_point,t_filling_rainy_point, t_filling_dry_point]
                    for var, var_point in zip(var_list, var_list_point):
                        if var_point.shape != (0,):
                            var.append(var_point)
        if x_test:
            x_train = np.concatenate(x_train, axis=0)
            y_train = np.concatenate(y_train, axis=0)
            y_train_true = np.concatenate(y_train_true, axis=0)
            x_test = np.concatenate(x_test, axis=0)
            y_test = np.concatenate(y_test, axis=0)
            y_test_true = np.concatenate(y_test_true, axis=0)
            x_filling = np.concatenate(x_filling, axis=0)

        # 将x_train/x_test/x_filling的所有nan值替换为0
        x_train = np.nan_to_num(x_train)
        x_test = np.nan_to_num(x_test)
        x_filling = np.nan_to_num(x_filling)

        return x_train, y_train, y_train_true, x_test, y_test, y_test_true,  x_filling, 
def area_split_random_without_filling(area_data,area_size, MinMaxNormalization_data, MinMaxNormalization_matrix, timestamps, methods_first="rainydry", methods_second="MinMax", time_step= 60):
    '''
    这个函数是随机划分数据集，不是按照时间顺序划分数据集，但是依然按照雨季还是非雨季划分数据集
    '''
    if methods_first == "rainydry":
        # 初始化地理网格的x数据集,每个的形状为：area_data.lon.shape[0],area_data.lat.shape[0]，~
        lon_size,lat_size = area_size,area_size
        x_train_rainy_grid, x_test_rainy_grid, x_train_dry_grid, x_test_dry_grid = np.empty((lon_size, lat_size), dtype=object),np.empty((lon_size, lat_size), dtype=object),np.empty((lon_size, lat_size), dtype=object),np.empty((lon_size, lat_size), dtype=object)
        y_true_train_rainy_grid, y_true_test_rainy_grid, y_true_train_dry_grid, y_true_test_dry_grid = np.empty((lon_size, lat_size), dtype=object), np.empty((lon_size, lat_size), dtype=object),np.empty((lon_size, lat_size), dtype=object),np.empty((lon_size, lat_size), dtype=object)
        
        # 初始化训练集和测试集
        x_train_rainy, y_train_rainy, y_train_true_rainy,  x_test_rainy, y_test_rainy, y_test_true_rainy,\
            x_train_dry, y_train_dry, y_train_true_dry,  x_test_dry, y_test_dry, y_test_true_dry,  = [], [], [], [], [], [],[], [], [],[], [], []
        
        var_list = [x_train_rainy, y_train_rainy, y_train_true_rainy,  x_test_rainy, y_test_rainy, y_test_true_rainy,\
                     x_train_dry, y_train_dry, y_train_true_dry,  x_test_dry, y_test_dry, y_test_true_dry]
        print(area_data.lon.shape[0])
        print(area_data.lat.shape[0])
        # 数据(MinMaxNormalization_data)划分为样本
        for i in range(area_data.lon.shape[0]):
            for j in range(area_data.lat.shape[0]):
                #如果目标变量在这个点的所有时间点上全为nan，就跳过这个点
                if area_data.isel(lon=i, lat=j)[target_var].isnull().all():
                    continue
                else:
                    point_data = MinMaxNormalization_data[:,i, j, :]
                    x_train_point_rainy, y_train_point_rainy, y_true_train_point_rainy, x_test_point_rainy, y_test_point_rainy, y_true_test_point_rainy, \
                        x_train_point_dry, y_train_point_dry, y_true_train_point_dry,  \
                            x_test_point_dry, y_test_point_dry, y_true_test_point_dry = \
                                    split_rainy_dry_data_random_without_filling(point_data, MinMaxNormalization_matrix[i,j,:], methods_second, timestamps, time_step)
                    
                    # 地理网格赋值
                    x_train_rainy_grid[i,j],x_test_rainy_grid[i,j], x_train_dry_grid[i,j], x_test_dry_grid[i,j] = \
                        np.nan_to_num(x_train_point_rainy), np.nan_to_num(x_test_point_rainy), np.nan_to_num(x_train_point_dry), np.nan_to_num(x_test_point_dry)
                    y_true_train_rainy_grid[i,j], y_true_test_rainy_grid[i,j], y_true_train_dry_grid[i,j], y_true_test_dry_grid[i,j] = \
                        y_true_train_point_rainy, y_true_test_point_rainy, y_true_train_point_dry, y_true_test_point_dry

                    var_list_point = [x_train_point_rainy, y_train_point_rainy, y_true_train_point_rainy,  x_test_point_rainy, y_test_point_rainy, \
                    y_true_test_point_rainy, x_train_point_dry, y_train_point_dry, y_true_train_point_dry, \
                    x_test_point_dry, y_test_point_dry, y_true_test_point_dry]
                    
                    # 判断输出的每一个数据，只有形状为(0,)的数据才append,为(0,)的数据跳过
                    # 使用for循环判断每一个数据是否为(0,)，如果是(0,)就跳过，不是(0,)就append
                    for var, var_point in zip(var_list, var_list_point):
                        if var_point.shape != (0,):
                            var.append(var_point)
        if x_test_rainy:
            '''
            # 使用循环判断
            for var in var_list:
                var = np.concatenate(var, axis=0)
            '''
            x_train_rainy = np.concatenate(x_train_rainy, axis=0)
            y_train_rainy = np.concatenate(y_train_rainy, axis=0)
            y_train_true_rainy = np.concatenate(y_train_true_rainy, axis=0)
            x_test_rainy = np.concatenate(x_test_rainy, axis=0)
            y_test_rainy = np.concatenate(y_test_rainy, axis=0)
            y_test_true_rainy = np.concatenate(y_test_true_rainy, axis=0)
            x_train_dry = np.concatenate(x_train_dry, axis=0)
            y_train_dry = np.concatenate(y_train_dry, axis=0)
            y_train_true_dry = np.concatenate(y_train_true_dry, axis=0)
            x_test_dry = np.concatenate(x_test_dry, axis=0)
            y_test_dry = np.concatenate(y_test_dry, axis=0)
            y_test_true_dry = np.concatenate(y_test_true_dry, axis=0)
        # 将x_train/x_test/x_filling的所有nan值替换为0
        x_train_rainy = np.nan_to_num(x_train_rainy)
        x_test_rainy = np.nan_to_num(x_test_rainy)
        x_train_dry = np.nan_to_num(x_train_dry)
        x_test_dry = np.nan_to_num(x_test_dry)
        return x_train_rainy, y_train_rainy, y_train_true_rainy, x_test_rainy, y_test_rainy, y_test_true_rainy, \
             x_train_dry, y_train_dry, y_train_true_dry, x_test_dry, y_test_dry, y_test_true_dry, \
                 x_train_rainy_grid, x_test_rainy_grid, x_train_dry_grid, x_test_dry_grid,\
                    y_true_train_rainy_grid, y_true_test_rainy_grid, y_true_train_dry_grid, y_true_test_dry_grid
    else:
        # 初始化训练集和测试集
        x_train, y_train, y_train_true, x_test, y_test, y_test_true = [], [], [], [], [], []
        var_list = [x_train, y_train, y_train_true, x_test, y_test, y_test_true]

        # 数据(MinMaxNormalization_data)划分为样本
        for i in range(area_data.lon.shape[0]):
            for j in range(area_data.lat.shape[0]):
                #如果目标变量在这个点的所有时间点上全为nan，就跳过这个点
                if area_data.isel(lon=i, lat=j)[target_var].isnull().all():
                    continue
                else:
                    point_data = MinMaxNormalization_data[:,i, j, :]
                    x_train_point, y_train_point, y_true_train_point,  x_test_point, y_test_point, y_true_test_point = split_regular_data(point_data,  MinMaxNormalization_matrix[i,j,:], methods_second, time_step)
                    var_list_point = [x_train_point, y_train_point, y_true_train_point, x_test_point, y_test_point, y_true_test_point]
                    for var, var_point in zip(var_list, var_list_point):
                        if var_point.shape != (0,):
                            var.append(var_point)
        if x_test:
            x_train = np.concatenate(x_train, axis=0)
            y_train = np.concatenate(y_train, axis=0)
            y_train_true = np.concatenate(y_train_true, axis=0)
            x_test = np.concatenate(x_test, axis=0)
            y_test = np.concatenate(y_test, axis=0)
            y_test_true = np.concatenate(y_test_true, axis=0)

        # 将x_train/x_test/x_filling的所有nan值替换为0
        x_train = np.nan_to_num(x_train)
        x_test = np.nan_to_num(x_test)

        return x_train, y_train, y_train_true, x_test, y_test, y_test_true
def area_split_random_without_filling(area_data, MinMaxNormalization_data, MinMaxNormalization_matrix, time_step=60, file_path=None):
    '''
    这个函数是随机划分数据集，不按照时间顺序划分数据集，也不考虑地理位置。
    '''
    # 初始化训练集和测试集
    x_train, y_train, y_train_true, x_test, y_test, y_test_true = [], [], [], [], [], []

    # 数据(MinMaxNormalization_data)划分为样本
    for i in range(area_data.lon.shape[0]):
        for j in range(area_data.lat.shape[0]):
            # 如果目标变量在这个点的所有时间点上全为nan，就跳过这个点
            if area_data.isel(lon=i, lat=j)[target_var].isnull().all():
                continue
            else:
                point_data = MinMaxNormalization_data[:,i, j, :]
                x_train_point, y_train_point, y_true_train_point, x_test_point, y_test_point, y_true_test_point = \
                        split_data_random_without_filling(point_data, MinMaxNormalization_matrix[i,j,:], time_step)
                
                # 将点数据添加到总数据集
                x_train += x_train_point
                y_train += y_train_point
                y_train_true += y_true_train_point
                x_test += x_test_point
                y_test += y_test_point
                y_test_true += y_true_test_point
    
    # 将列表转换为numpy数组
    x_train = np.array(x_train)
    y_train = np.array(y_train)
    y_train_true = np.array(y_train_true)
    x_test = np.array(x_test)
    y_test = np.array(y_test)
    y_test_true = np.array(y_test_true)
    
    # 分批次保存数据
    batch_size = 50
    for start_idx in range(0, len(x_train), batch_size):
        end_idx = min(start_idx + batch_size, len(x_train))
        np.save(file_path + f"x_train_batch_{start_idx // batch_size}.npy", x_train[start_idx:end_idx])
        np.save(file_path + f"y_train_batch_{start_idx // batch_size}.npy", y_train[start_idx:end_idx])
        np.save(file_path + f"y_train_true_batch_{start_idx // batch_size}.npy", y_train_true[start_idx:end_idx])

    for start_idx in range(0, len(x_test), batch_size):
        end_idx = min(start_idx + batch_size, len(x_test))
        np.save(file_path + f"x_test_batch_{start_idx // batch_size}.npy", x_test[start_idx:end_idx])
        np.save(file_path + f"y_test_batch_{start_idx // batch_size}.npy", y_test[start_idx:end_idx])
        np.save(file_path + f"y_test_true_batch_{start_idx // batch_size}.npy", y_test_true[start_idx:end_idx])

    return x_train, y_train, y_train_true, x_test, y_test, y_test_true
# 模型定义、回调函数、模型训练
def train_model(x_train_rainy, y_train_rainy, TIME_STEPS, INPUT_DIMS, drop):
    model = LSTM_model(TIME_STEPS, INPUT_DIMS, drop)
    initial_learning_rate = 1e-4
    model.compile(optimizer=Adam(learning_rate=initial_learning_rate), loss='mae', metrics=['mse', 'mae'])
    # 设置停止阈
    early_stopping = EarlyStopping(monitor='loss',
                                patience=3,
                                min_delta=1e-4,
                                mode='auto',
                                restore_best_weights=True,#是否从具有监测数量的最佳值的时期恢复模型权重
                                verbose=5)
    history =  model.fit(x_train_rainy, y_train_rainy, epochs=50, batch_size=32, validation_split=0.2,callbacks=[early_stopping])
    return model, history
# LSTM模型
def LSTM_model(window_length, input_dim, dropout_rate=0.25, batch_size=None):
    model = Sequential()
    model.add(Masking(mask_value=0, input_shape=(window_length, input_dim)))
    model.add(LSTM(units=204,input_shape=(window_length, input_dim), return_sequences=True))
    model.add(Dropout(dropout_rate))
    model.add(LSTM(units=116, return_sequences=False))
    model.add(Dense(1))
    return model
# 学习率调整函数
def scheduler(epoch):
    # 每隔5个epoch，学习率减小为原来的1/10
    if epoch % 5== 0 and epoch != 0:
        lr = K.get_value(model.optimizer.lr)
        if lr>1e-6:
            K.set_value(model.optimizer.lr, lr * 0.1)
            print("lr changed to {}".format(lr * 0.1))
    return K.get_value(model.optimizer.lr)
reduce_lr = LearningRateScheduler(scheduler)
# 模型预测和评估
def predict_point(point_data, normalize_data, model):
    '''
    tips:
    1. Val_loss和真实的预测数据有区别，这个是没有逆归一化后的数据大小，因此可能会非常大 
    2. 逆归一化的数值结果只和 normalize_data数据相关，和预测数据相关性无关
    单点预测函数，使用经纬度提取单点数据，划分数据集，返回预测结果
    point_data:x的单点数据，用于预测未逆归一化的y值
    normalize_data:归一化数据，用于逆归一化
    model:训练好的模型
    '''
    # 预测
    y_pre = model.predict(point_data)
    # 逆归一化
    y_pre_inverse = IMinMaxNormalization(y_pre,normalize_data)
    return y_pre_inverse
def predict_area(area_data,area_size, area_x_data, normalization_matrix, model):
    '''
    逐点预测和逆归一化,返回的grid是区域数据，每个点是一个list
    '''
    y_pre_inverse_grid = np.empty((area_size, area_size), dtype=object)
    y_pre_inverse = []
    lon_size,lat_size = area_data.lon.shape[0],area_data.lat.shape[0]
    for i in range(lon_size):
        for j in range(lat_size):
            # 如果目标变量在这个点的所有时间点上全为nan，就跳过这个点
            if area_data.isel(lon=i, lat=j)[target_var].isnull().all() or area_x_data[i,j].shape[0] == 0:
                continue
            else:
                y_pre_inverse_temp = predict_point(area_x_data[i,j], normalization_matrix[i,j,:], model)
                y_pre_inverse_temp = np.array(y_pre_inverse_temp[:,0])
                y_pre_inverse_grid[i,j] = y_pre_inverse_temp
                y_pre_inverse.append(y_pre_inverse_temp)
    if len(y_pre_inverse) > 0:
        y_pre_inverse = np.concatenate(y_pre_inverse, axis=0)
    else:
        y_pre_inverse = np.array([])
    return y_pre_inverse, y_pre_inverse_grid
def evaluate(y_true, y_pre):
    # 计算RMSE
    rmse = np.sqrt(mean_squared_error(y_true, y_pre))
    print(f'Root Mean Squared Error (RMSE): {rmse}')
    # R2决定系数
    r2 = r2_score(y_true, y_pre)
    print(f'R-squared (R2): {r2}')
    # 平均绝对误差MAE
    mae = mean_absolute_error(y_true, y_pre)
    print(f'Mean Absolute Error (MAE): {mae}')
    return rmse, r2, mae
def evaluate_grid(y_true_grid, y_pre_grid):
    rmse_grid,mae_grid,r2_grid = np.empty((y_true_grid.shape[0],y_true_grid.shape[1],y_true_grid.shape[2])),np.empty((y_true_grid.shape[0],y_true_grid.shape[1],y_true_grid.shape[2])),np.empty((y_true_grid.shape[0],y_true_grid.shape[1],y_true_grid.shape[2]))
    for i in range(y_true_grid.shape[0]):
        for j in range(y_true_grid.shape[1]):
            for k in range(y_true_grid.shape[2]):
                if y_true_grid[i,j,k].shape[0]==0 or y_pre_grid[i,j,k].shape[0]==0:
                    rmse_grid[i,j,k],mae_grid[i,j,k],r2_grid[i,j,k] = np.nan,np.nan,np.nan
                    continue
                rmse_grid[i,j,k],mae_grid[i,j,k],r2_grid[i,j,k] = evaluate(y_true_grid[i,j,k], y_pre_grid[i,j,k])
    return rmse_grid,mae_grid,r2_grid
def model_save(file_path, model, history,str):
    #保存模型
    model.save(os.path.join(file_path, str+"_model.h5"))

### <font face="宋体" size=4 color=#A52A2A>常量准备

In [3]:
def main(args):
    model_argu = load_config(args.model_config_path)
    missing_values = load_config(args.vars_config_path)
    data_argu = load_config(args.data_process_config_path)
    path_argu = load_config(args.path_config_path)
    var_list = list(missing_values.keys())
    return model_argu, missing_values, var_list ,data_argu, path_argu

if __name__ == '__main__':
    parser = argparse.ArgumentParser()
    parser.add_argument('--model_config_path', type=str, default='./config/model_config.yaml')
    parser.add_argument('--vars_config_path', type=str, default='./config/vars_config.yaml')
    parser.add_argument('--data_process_config_path', type=str, default='./config/data_process_config.yaml')
    parser.add_argument('--path_config_path', type=str, default='./config/path_config.yaml')

    args = parser.parse_known_args()[0]
    model_argu, missing_values, var_list ,data_argu, path_argu = main(args)
    data = data_preparation(path_argu['SMAP_file_path'],var_list[0],var_list,missing_values)

    # 定义时间步长(2个月)
    time_step = 60
    timestamps = np.arange(data.time.shape[0])
    # 定义单幅尺寸,研究区尺寸为area_size*area_size
    area_size = 10
    # 定义研究区域经纬度尺寸
    lon_shape = data.lon.shape[0]
    lat_shape = data.lat.shape[0]

    lon_num = lon_shape//area_size+1
    lat_num = lat_shape//area_size+1


    # 设置训练参数
    TIME_STEPS = time_step
    INPUT_DIMS = len(var_list)
    lstm_units = 64
    drop = 0.25

    # 经纬度
    lon_min = data.lon.min()
    lon_max = data.lon.max()
    lat_min = data.lat.min()
    lat_max = data.lat.max()

In [8]:
data

<xarray.Dataset>
Dimensions:                               (time: 8767, lat: 42, lon: 104)
Coordinates:
  * time                                  (time) datetime64[ns] 2016-01-01T03...
  * lat                                   (lat) float32 25.29 25.67 ... 40.6
  * lon                                   (lon) float32 66.85 67.22 ... 105.3
Data variables: (12/42)
    soil_moisture                         (time, lat, lon) float64 nan ... nan
    hourlyPrecipRateGC                    (time, lat, lon) float32 0.0 ... 0.0
    as_dem_3s                             (time, lat, lon) float32 115.2 ... ...
    Aspect1                               (time, lat, lon) float32 193.1 ... ...
    Slope                                 (time, lat, lon) float32 89.72 ... ...
    sand_0-5cm_mean                       (time, lat, lon) float32 458.6 ... ...
    ...                                    ...
    Albedo_WSA_Band7                      (time, lat, lon) float32 334.0 ... ...
    NDVI                                  (time, lat, lon) float32 1.029e+03 ...
    EVI                                   (time, lat, lon) float32 722.5 ... ...
    Longitude                             (time, lat, lon) float64 66.85 ... ...
    Latitude                              (time, lat, lon) float64 25.29 ... ...
    DOY                                   (time, lat, lon) float64 3.0 ... 8....

### <font face="宋体" size=4 color=#A52A2A>训练开始

In [22]:
# 分为粗和细粒度分辨率
lon_shape_cor,lat_shape_cor = lon_num*area_size,lat_num*area_size
test_rmse_grid,test_mae_grid,test_r2_grid = np.empty((lon_shape_cor, lat_shape_cor,2), dtype=object),np.empty((lon_shape_cor, lat_shape_cor,2), dtype=object),np.empty((lon_shape_cor, lat_shape_cor,2), dtype=object)
train_rmse_grid,train_mae_grid,train_r2_grid = np.empty((lon_shape_cor, lat_shape_cor,2), dtype=object),np.empty((lon_shape_cor, lat_shape_cor,2), dtype=object),np.empty((lon_shape_cor, lat_shape_cor,2), dtype=object)

# 大区域样本预测值，大区域样本实际值
test_y_pred,test_y_true = np.empty((lon_shape_cor, lat_shape_cor,2), dtype=object),np.empty((lon_shape_cor, lat_shape_cor,2), dtype=object)
train_y_pred,train_y_true = np.empty((lon_shape_cor, lat_shape_cor,2), dtype=object),np.empty((lon_shape_cor, lat_shape_cor,2), dtype=object)

folder_path = r"C:\Users\Administrator\Desktop\code\LSTM\SMAP_ERA5_Data\all_exp\10_model"
# 循环返回粗和细粒度分辨率矩阵用于绘制误差图
for i in range(lon_num):
    for j in range(lat_num):
        # 文件存在
        if i > (lon_shape//area_size) or j > (lat_shape//area_size) or os.path.exists(os.path.join(folder_path, f"lon{i}_lat{j}_rainy_model.h5")):
            print(f"lon{i}_lat{j}_rainy_model.h5 extenced")
            continue
        else:
            # 获取区域数据
            area_data = select_data(data,i+1,j+1,area_size)
            # 数据掩码和掩码矩阵生成
            masked_area_dataset,mask_array = mask_fun(area_data,missing_values,var_list)
            # 数据标准化
            MinMaxNormalization_data, MinMaxNormalization_matrix = MinMaxNormalization(masked_area_dataset)
            # 数据划分
            x_train_rainy, y_train_rainy, y_train_true_rainy, x_test_rainy, y_test_rainy, y_test_true_rainy,\
            x_train_dry, y_train_dry, y_train_true_dry, x_test_dry, y_test_dry, y_test_true_dry,\
            x_train_rainy_grid, x_test_rainy_grid, x_train_dry_grid, x_test_dry_grid,\
            y_true_train_rainy_grid, y_true_test_rainy_grid, y_true_train_dry_grid, y_true_test_dry_grid = \
            area_split_data_without_filling(area_data, area_size, MinMaxNormalization_data, MinMaxNormalization_matrix, timestamps, methods_first=methods_first, methods_second=methods_second, time_step=time_step)
            # 如不存在训练数据，跳过
            if x_train_rainy.shape[0] == 0 or x_train_dry.shape[0] == 0:
                continue
            # 训练模型
            model_rainy, history_rainy= train_model(x_train_rainy, y_train_rainy, time_step, x_train_rainy.shape[-1], drop)
            model_dry,  history_dry= train_model(x_train_dry, y_train_dry, time_step, x_train_dry.shape[-1], drop)
            # 保存模型
            file_model_path = r"C:\Users\Administrator\Desktop\code\LSTM\SMAP_ERA5_Data\all_exp\10_model"
            rainy_str = f"lon{i}_lat{j}_rainy"
            dry_str = f"lon{i}_lat{j}_dry"
            model_save(file_model_path, model_rainy, history_rainy, rainy_str)
            model_save(file_model_path, model_dry, history_dry, dry_str)
                        # 区域预测
            y_test_pre_inverse_rainy, y_test_pre_inverse_grid_rainy = predict_area(area_data,area_size, x_test_rainy_grid,MinMaxNormalization_matrix, model_rainy)
            y_test_pre_inverse_dry, y_test_pre_inverse_grid_dry = predict_area(area_data,area_size, x_test_dry_grid, MinMaxNormalization_matrix, model_dry)
            y_train_pre_inverse_rainy, y_train_pre_inverse_grid_rainy = predict_area(area_data,area_size, x_train_rainy_grid, MinMaxNormalization_matrix, model_rainy)
            y_train_pre_inverse_dry, y_train_pre_inverse_grid_dry = predict_area(area_data,area_size, x_train_dry_grid, MinMaxNormalization_matrix, model_dry)

            # 评估
            test_rmse_rainy,test_r2_rainy,test_mae_rainy = evaluate(y_test_true_rainy, y_test_pre_inverse_rainy)
            test_rmse_dry,test_r2_dry,test_mae_dry = evaluate(y_test_true_dry, y_test_pre_inverse_dry)
            train_rmse_rainy, train_r2_rainy, train_mae_rainy = evaluate(y_train_true_rainy, y_train_pre_inverse_rainy)
            train_rmse_dry, train_r2_dry, train_mae_dry = evaluate(y_train_true_dry, y_train_pre_inverse_dry)

            # 逐个预测
            test_y_pred[i*area_size:(i+1)*area_size,j*area_size:(j+1)*area_size,:],train_y_pred[i*area_size:(i+1)*area_size,j*area_size:(j+1)*area_size,:] = np.stack((y_test_pre_inverse_grid_rainy,y_test_pre_inverse_grid_dry),axis=-1),np.stack((y_train_pre_inverse_grid_rainy,y_train_pre_inverse_grid_dry),axis=-1)
            test_y_true[i*area_size:(i+1)*area_size,j*area_size:(j+1)*area_size,:],train_y_true[i*area_size:(i+1)*area_size,j*area_size:(j+1)*area_size,:] = np.stack((y_true_test_rainy_grid,y_true_test_dry_grid),axis=-1),np.stack((y_true_train_rainy_grid,y_true_train_dry_grid),axis=-1)
            
            # 将grid数据以一块一块的形式拼接到大块中
            test_rmse_grid[i*area_size:(i+1)*area_size,j*area_size:(j+1)*area_size,:],test_mae_grid[i*area_size:(i+1)*area_size,j*area_size:(j+1)*area_size,:],test_r2_grid[i*area_size:(i+1)*area_size,j*area_size:(j+1)*area_size,:] = evaluate_grid(np.stack((y_true_test_rainy_grid,y_true_test_dry_grid),axis=2),np.stack((y_test_pre_inverse_grid_rainy,y_test_pre_inverse_grid_dry),axis=2))
            train_rmse_grid[i*area_size:(i+1)*area_size,j*area_size:(j+1)*area_size,:],train_mae_grid[i*area_size:(i+1)*area_size,j*area_size:(j+1)*area_size,:],train_r2_grid[i*area_size:(i+1)*area_size,j*area_size:(j+1)*area_size,:] = evaluate_grid(np.stack((y_true_train_rainy_grid,y_true_train_dry_grid),axis=2),np.stack((y_train_pre_inverse_grid_rainy,y_train_pre_inverse_grid_dry),axis=2))

            # 填充数据预测
            test_rmse[i,j,:],test_mae[i,j,:],test_r2[i,j,:] = np.stack((test_rmse_rainy,test_rmse_dry),axis=-1),np.stack((test_mae_rainy,test_mae_dry),axis=-1),np.stack((test_r2_rainy,test_r2_dry),axis=-1)
            train_rmse[i,j,:],train_mae[i,j,:],train_r2[i,j,:] = np.stack((train_rmse_rainy,train_rmse_dry),axis=-1),np.stack((train_mae_rainy,train_mae_dry),axis=-1),np.stack((train_r2_rainy,train_r2_dry),axis=-1)


# 初始化列表以收集所有点的数据
test_y_pred_all_rainy, test_y_true_all_rainy = [], []
test_y_pred_all_dry, test_y_true_all_dry = [], []
train_y_pred_all_rainy, train_y_true_all_rainy = [], []
train_y_pred_all_dry, train_y_true_all_dry = [], []

for i in range(lon_shape_cor):
    for j in range(lat_shape_cor):
        # 对于每个点，确保数据存在，然后添加到列表中
        if test_y_pred[i, j, 0] is not None and test_y_pred[i, j, 0].size != 0:
            test_y_pred_all_rainy.append(test_y_pred[i, j, 0])
            test_y_true_all_rainy.append(test_y_true[i, j, 0])
        if test_y_pred[i, j, 1] is not None and test_y_pred[i, j, 1].size != 0:
            test_y_pred_all_dry.append(test_y_pred[i, j, 1])
            test_y_true_all_dry.append(test_y_true[i, j, 1])

        if train_y_pred[i, j, 0] is not None and train_y_pred[i, j, 0].size != 0:
            train_y_pred_all_rainy.append(train_y_pred[i, j, 0])
            train_y_true_all_rainy.append(train_y_true[i, j, 0])
        if train_y_pred[i, j, 1] is not None and train_y_pred[i, j, 1].size != 0:
            train_y_pred_all_dry.append(train_y_pred[i, j, 1])
            train_y_true_all_dry.append(train_y_true[i, j, 1])

# 使用numpy.concatenate连接所有点的数据
test_y_pred_concat_rainy = np.concatenate(test_y_pred_all_rainy)
test_y_true_concat_rainy = np.concatenate(test_y_true_all_rainy)
test_y_pred_concat_dry = np.concatenate(test_y_pred_all_dry)
test_y_true_concat_dry = np.concatenate(test_y_true_all_dry)

train_y_pred_concat_rainy = np.concatenate(train_y_pred_all_rainy)
train_y_true_concat_rainy = np.concatenate(train_y_true_all_rainy)
train_y_pred_concat_dry = np.concatenate(train_y_pred_all_dry)
train_y_true_concat_dry = np.concatenate(train_y_true_all_dry)

# 计算全域内的误差
test_rmse_all_rainy, test_mae_all_rainy, test_r2_all_rainy = evaluate(test_y_true_concat_rainy, test_y_pred_concat_rainy)
test_rmse_all_dry, test_mae_all_dry, test_r2_all_dry = evaluate(test_y_true_concat_dry, test_y_pred_concat_dry)
train_rmse_all_rainy, train_mae_all_rainy, train_r2_all_rainy = evaluate(train_y_true_concat_rainy, train_y_pred_concat_rainy)
train_rmse_all_dry, train_mae_all_dry, train_r2_all_dry = evaluate(train_y_true_concat_dry, train_y_pred_concat_dry)

data_path = r"C:\Users\Administrator\Desktop\code\LSTM\SMAP_ERA5_Data\all_exp\10_data"
# 保存误差变量
save_list = [test_rmse, test_mae, test_r2, train_rmse, train_mae, train_r2, test_rmse_grid, test_mae_grid, test_r2_grid, train_rmse_grid, train_mae_grid, train_r2_grid,
             test_rmse_all_rainy, test_mae_all_rainy, test_r2_all_rainy, test_rmse_all_dry, test_mae_all_dry, test_r2_all_dry, train_rmse_all_rainy, train_mae_all_rainy, train_r2_all_rainy, train_rmse_all_dry, train_mae_all_dry, train_r2_all_dry]

variables_dict = {
    "test_rmse": test_rmse,"test_mae": test_mae,"test_r2": test_r2,"train_rmse": train_rmse,"train_mae": train_mae,"train_r2": train_r2,\
    "test_rmse_grid": test_rmse_grid,"test_mae_grid": test_mae_grid,"test_r2_grid": test_r2_grid,"train_rmse_grid": train_rmse_grid,"train_mae_grid": train_mae_grid,"train_r2_grid": train_r2_grid,\
    "test_rmse_all_rainy": test_rmse_all_rainy,"test_mae_all_rainy": test_mae_all_rainy,"test_r2_all_rainy": test_r2_all_rainy,"test_rmse_all_dry": test_rmse_all_dry,"test_mae_all_dry": test_mae_all_dry,"test_r2_all_dry": test_r2_all_dry,\
    "train_rmse_all_rainy": train_rmse_all_rainy,"train_mae_all_rainy": train_mae_all_rainy,"train_r2_all_rainy": train_r2_all_rainy,"train_rmse_all_dry": train_rmse_all_dry,"train_mae_all_dry": train_mae_all_dry,"train_r2_all_dry": train_r2_all_dry
}
# 所有以save_list变量名保存
for var_name, var_data in variables_dict.items():
    file_path = os.path.join(data_path, var_name + ".npy")
    np.save(file_path, var_data)

lon0_lat0_rainy_model.h5 extenced
lon0_lat1_rainy_model.h5 extenced
lon0_lat2_rainy_model.h5 extenced
lon0_lat3_rainy_model.h5 extenced
lon1_lat0_rainy_model.h5 extenced
lon1_lat1_rainy_model.h5 extenced
lon1_lat2_rainy_model.h5 extenced
Error:lon_num or lat_num is too large


C:\Users\Administrator\AppData\Local\Temp\ipykernel_54420\2381953606.py:83: RuntimeWarning: All-NaN slice encountered
  min_vals = np.nanmin(data, axis=0)
C:\Users\Administrator\AppData\Local\Temp\ipykernel_54420\2381953606.py:84: RuntimeWarning: All-NaN slice encountered
  max_vals = np.nanmax(data, axis=0)


Epoch 1/50
643/643 [==============================] - 24s 22ms/step - loss: 0.1245 - mse: 0.0283 - mae: 0.1245 - val_loss: 0.0917 - val_mse: 0.0182 - val_mae: 0.0917
Epoch 2/50
643/643 [==============================] - 11s 17ms/step - loss: 0.1071 - mse: 0.0217 - mae: 0.1071 - val_loss: 0.0884 - val_mse: 0.0172 - val_mae: 0.0884
Epoch 3/50
643/643 [==============================] - 11s 17ms/step - loss: 0.0973 - mse: 0.0186 - mae: 0.0973 - val_loss: 0.0903 - val_mse: 0.0166 - val_mae: 0.0903
Epoch 4/50
643/643 [==============================] - 11s 17ms/step - loss: 0.0898 - mse: 0.0163 - mae: 0.0898 - val_loss: 0.0921 - val_mse: 0.0159 - val_mae: 0.0921
Epoch 5/50
643/643 [==============================] - 11s 18ms/step - loss: 0.0850 - mse: 0.0150 - mae: 0.0850 - val_loss: 0.0817 - val_mse: 0.0166 - val_mae: 0.0817
Epoch 6/50
643/643 [==============================] - 12s 19ms/step - loss: 0.0824 - mse: 0.0144 - mae: 0.0824 - val_loss: 0.0849 - val_mse: 0.0151 - val_mae: 0.0849
Epoc

C:\Users\Administrator\AppData\Local\Temp\ipykernel_54420\2381953606.py:83: RuntimeWarning: All-NaN slice encountered
  min_vals = np.nanmin(data, axis=0)
C:\Users\Administrator\AppData\Local\Temp\ipykernel_54420\2381953606.py:84: RuntimeWarning: All-NaN slice encountered
  max_vals = np.nanmax(data, axis=0)


Epoch 1/50
914/914 [==============================] - 30s 22ms/step - loss: 0.0998 - mse: 0.0227 - mae: 0.0998 - val_loss: 0.0656 - val_mse: 0.0115 - val_mae: 0.0656
Epoch 2/50
914/914 [==============================] - 17s 18ms/step - loss: 0.0840 - mse: 0.0175 - mae: 0.0840 - val_loss: 0.0593 - val_mse: 0.0109 - val_mae: 0.0593
Epoch 3/50
914/914 [==============================] - 17s 18ms/step - loss: 0.0802 - mse: 0.0162 - mae: 0.0802 - val_loss: 0.0583 - val_mse: 0.0106 - val_mae: 0.0583
Epoch 4/50
914/914 [==============================] - 17s 18ms/step - loss: 0.0763 - mse: 0.0153 - mae: 0.0763 - val_loss: 0.0732 - val_mse: 0.0118 - val_mae: 0.0732
Epoch 5/50
914/914 [==============================] - 17s 18ms/step - loss: 0.0740 - mse: 0.0146 - mae: 0.0740 - val_loss: 0.0522 - val_mse: 0.0101 - val_mae: 0.0522
Epoch 6/50
914/914 [==============================] - 17s 18ms/step - loss: 0.0724 - mse: 0.0140 - mae: 0.0724 - val_loss: 0.0619 - val_mse: 0.0103 - val_mae: 0.0619
Epoc

...

Epoch 1/50
1027/1027 [==============================] - 32s 22ms/step - loss: 0.1047 - mse: 0.0184 - mae: 0.1047 - val_loss: 0.1221 - val_mse: 0.0236 - val_mae: 0.1221
Epoch 2/50
1027/1027 [==============================] - 19s 18ms/step - loss: 0.0931 - mse: 0.0145 - mae: 0.0931 - val_loss: 0.1177 - val_mse: 0.0226 - val_mae: 0.1177
Epoch 3/50
1027/1027 [==============================] - 18s 18ms/step - loss: 0.0914 - mse: 0.0141 - mae: 0.0914 - val_loss: 0.1164 - val_mse: 0.0229 - val_mae: 0.1164
Epoch 4/50
1027/1027 [==============================] - 18s 18ms/step - loss: 0.0888 - mse: 0.0133 - mae: 0.0888 - val_loss: 0.1131 - val_mse: 0.0212 - val_mae: 0.1131
Epoch 5/50
1027/1027 [==============================] - 18s 18ms/step - loss: 0.0873 - mse: 0.0129 - mae: 0.0873 - val_loss: 0.1194 - val_mse: 0.0246 - val_mae: 0.1194
Epoch 6/50
1027/1027 [==============================] - 18s 18ms/step - loss: 0.0861 - mse: 0.0126 - mae: 0.0861 - val_loss: 0.1077 - val_mse: 0.0197 - val_mae:

C:\Users\Administrator\AppData\Local\Temp\ipykernel_54420\2381953606.py:83: RuntimeWarning: All-NaN slice encountered
  min_vals = np.nanmin(data, axis=0)
C:\Users\Administrator\AppData\Local\Temp\ipykernel_54420\2381953606.py:84: RuntimeWarning: All-NaN slice encountered
  max_vals = np.nanmax(data, axis=0)


Epoch 1/50
1113/1113 [==============================] - 34s 21ms/step - loss: 0.1035 - mse: 0.0191 - mae: 0.1035 - val_loss: 0.1140 - val_mse: 0.0257 - val_mae: 0.1140
Epoch 2/50
1113/1113 [==============================] - 20s 18ms/step - loss: 0.0897 - mse: 0.0150 - mae: 0.0897 - val_loss: 0.1139 - val_mse: 0.0235 - val_mae: 0.1139
Epoch 3/50
1113/1113 [==============================] - 20s 18ms/step - loss: 0.0865 - mse: 0.0142 - mae: 0.0865 - val_loss: 0.1043 - val_mse: 0.0225 - val_mae: 0.1043
Epoch 4/50
1113/1113 [==============================] - 20s 18ms/step - loss: 0.0844 - mse: 0.0136 - mae: 0.0844 - val_loss: 0.1018 - val_mse: 0.0233 - val_mae: 0.1018
Epoch 5/50
1113/1113 [==============================] - 20s 18ms/step - loss: 0.0825 - mse: 0.0131 - mae: 0.0825 - val_loss: 0.1028 - val_mse: 0.0203 - val_mae: 0.1028
Epoch 6/50
1113/1113 [==============================] - 20s 18ms/step - loss: 0.0807 - mse: 0.0126 - mae: 0.0807 - val_loss: 0.0958 - val_mse: 0.0201 - val_mae:

C:\Users\Administrator\AppData\Local\Temp\ipykernel_54420\2381953606.py:83: RuntimeWarning: All-NaN slice encountered
  min_vals = np.nanmin(data, axis=0)
C:\Users\Administrator\AppData\Local\Temp\ipykernel_54420\2381953606.py:84: RuntimeWarning: All-NaN slice encountered
  max_vals = np.nanmax(data, axis=0)


Epoch 1/50
799/799 [==============================] - 28s 23ms/step - loss: 0.0997 - mse: 0.0242 - mae: 0.0997 - val_loss: 0.0879 - val_mse: 0.0229 - val_mae: 0.0879
Epoch 2/50
799/799 [==============================] - 15s 19ms/step - loss: 0.0882 - mse: 0.0209 - mae: 0.0882 - val_loss: 0.0864 - val_mse: 0.0213 - val_mae: 0.0864
Epoch 3/50
799/799 [==============================] - 15s 19ms/step - loss: 0.0845 - mse: 0.0200 - mae: 0.0845 - val_loss: 0.0821 - val_mse: 0.0223 - val_mae: 0.0821
Epoch 4/50
799/799 [==============================] - 15s 19ms/step - loss: 0.0826 - mse: 0.0195 - mae: 0.0826 - val_loss: 0.0816 - val_mse: 0.0220 - val_mae: 0.0816
Epoch 5/50
799/799 [==============================] - 15s 18ms/step - loss: 0.0811 - mse: 0.0190 - mae: 0.0811 - val_loss: 0.0818 - val_mse: 0.0221 - val_mae: 0.0818
Epoch 6/50
799/799 [==============================] - 15s 19ms/step - loss: 0.0793 - mse: 0.0186 - mae: 0.0793 - val_loss: 0.0784 - val_mse: 0.0206 - val_mae: 0.0784
Epoc

### <font face="宋体" size=4 color=#A52A2A>变量保存

In [ ]:
import pickle
import os

# 假设变量存储在一个字典中，键是变量名，值是变量的值
save_list = [test_rmse, test_mae, test_r2, train_rmse, train_mae, train_r2, test_rmse_grid, test_mae_grid, test_r2_grid, train_rmse_grid, train_mae_grid, train_r2_grid,
             test_rmse_all_rainy, test_mae_all_rainy, test_r2_all_rainy, test_rmse_all_dry, test_mae_all_dry, test_r2_all_dry, train_rmse_all_rainy, train_mae_all_rainy, train_r2_all_rainy, train_rmse_all_dry, train_mae_all_dry, train_r2_all_dry]

variables = {
    "test_rmse": test_rmse,"test_mae": test_mae,"test_r2": test_r2,"train_rmse": train_rmse,"train_mae": train_mae,"train_r2": train_r2,\
    "test_rmse_grid": test_rmse_grid,"test_mae_grid": test_mae_grid,"test_r2_grid": test_r2_grid,"train_rmse_grid": train_rmse_grid,"train_mae_grid": train_mae_grid,"train_r2_grid": train_r2_grid,\
    "test_rmse_all_rainy": test_rmse_all_rainy,"test_mae_all_rainy": test_mae_all_rainy,"test_r2_all_rainy": test_r2_all_rainy,"test_rmse_all_dry": test_rmse_all_dry,"test_mae_all_dry": test_mae_all_dry,"test_r2_all_dry": test_r2_all_dry,\
    "train_rmse_all_rainy": train_rmse_all_rainy,"train_mae_all_rainy": train_mae_all_rainy,"train_r2_all_rainy": train_r2_all_rainy,"train_rmse_all_dry": train_rmse_all_dry,"train_mae_all_dry": train_mae_all_dry,"train_r2_all_dry": train_r2_all_dry
}
# 定义保存变量的目录,创建saved_variables文件夹即可
save_path = r"C:\Users\Administrator\Desktop\code\LSTM\SMAP_ERA5_Data\all_exp\10_data\saved_variables"
os.makedirs(save_path, exist_ok=True)

# 保存每个变量到单独的文件
for var_name, var_value in variables.items():
    with open(os.path.join(save_path, var_name + '.pkl'), 'wb') as file:
        pickle.dump(var_value, file)

### <font face="宋体" size=4 color=#A52A2A>变量加载

In [ ]:
import glob

# 定义要加载变量的目录
load_dir = r"C:\Users\Administrator\Desktop\code\LSTM\SMAP_ERA5_Data\all_exp\10_data\saved_variables"

# 读取目录中的所有pickle文件
for file_path in glob.glob(os.path.join(load_dir, '*.pkl')):
    with open(file_path, 'rb') as file:
        # 获取变量名（文件名）
        var_name = os.path.basename(file_path).split('.')[0]
        # 加载变量
        globals()[var_name] = pickle.load(file)
        print(f"Loaded {var_name}")


### <font face="宋体" size=4 color=#A52A2A>数据填充，使用之前训练得到的模型将所有数据填充完整

In [23]:
# 此块的大致思想：分块进行，分块加载模型，分块预测
model_file_path = r"C:\Users\Administrator\Desktop\code\LSTM\SMAP_ERA5_Data\all_exp\10_model"
y_pre_inverse_grid = np.empty((data.time.shape[0]-60,data.lon.shape[0],data.lat.shape[0]), dtype=object)
def combine_data(data_rainy, data_dry, rainy_ind, dry_ind):
    '''
    使用ind来整合两个rainy和dry数据,每个数据都是二维矩阵，每个点里面都是一个含有数据的list
    '''
    len_time = len(data_rainy[0, 0]) + len(data_dry[0, 0])
    lon, lat = data_rainy.shape
    return_data = np.empty((len_time, lon, lat), dtype=object)

    for i in range(lon):
        for j in range(lat):
            data_point = np.empty(len_time, dtype=object)
            index = 0
            for rainy_index in rainy_ind[i, j]:
                data_point[int(rainy_index)] = data_rainy[i, j][index]
                index += 1
            index = 0
            for dry_index in dry_ind[i, j]:
                data_point[int(dry_index)] = data_dry[i, j][index]
                index += 1
            return_data[:, i, j] = np.array(data_point)

    return return_data

def predict_area_filling(area_x_data, normalization_matrix, model):
    '''
    逐点预测和逆归一化,返回的grid是区域数据，每个点是一个list
    '''
    lon,lat = area_x_data.shape[0],area_x_data.shape[1]
    y_pre_inverse_grid = np.empty((lon, lat), dtype=object)
    for i in range(lon):
        for j in range(lat):
            if area_x_data[i,j].shape[0] == 0:
                continue
            else:
                y_pre_inverse_temp = predict_point(area_x_data[i,j], normalization_matrix[i,j,:], model)
                y_pre_inverse_temp = np.array(y_pre_inverse_temp[:,0])
                y_pre_inverse_grid[i,j] = y_pre_inverse_temp
    return y_pre_inverse_grid

for i in range(lon_num):
    for j in range(lat_num):
        #文件名命名方式为：lon0_lat0_dry_model  提取区域的干湿模型
        dry_str = f"lon{i}_lat{j}_dry_model"
        rainy_str = f"lon{i}_lat{j}_rainy_model"
        #判断模型是否存在
        if os.path.exists(os.path.join(model_file_path, dry_str+".h5")) and os.path.exists(os.path.join(model_file_path, rainy_str+".h5")):
            #加载模型
            model_dry = load_model(os.path.join(model_file_path, dry_str+".h5"))
            model_rainy = load_model(os.path.join(model_file_path, rainy_str+".h5"))
            # 获取区域数据
            area_data = select_data(data,i+1,j+1,area_size)
            # 数据掩码和掩码矩阵生成
            masked_area_dataset,mask_array = mask_fun(area_data,missing_values,var_list)
            # 数据标准化
            MinMaxNormalization_data, MinMaxNormalization_matrix = MinMaxNormalization(masked_area_dataset)
            # 数据划分(data,missing_values,var_sets,normal_methods,timestamps,TIME_STEPS,area_size):
            x_rainy_filling_grid, x_dry_filling_grid,x_rainy_ind_grid, x_dry_ind_grid = area_split_data_to_filling(masked_area_dataset,TIME_STEPS)
            # 数据预测(area_x_data, normalization_matrix, model):
            y_pre_inverse_grid_rainy = predict_area_filling(x_rainy_filling_grid, MinMaxNormalization_matrix, model_rainy)
            y_pre_inverse_grid_dry = predict_area_filling(x_dry_filling_grid, MinMaxNormalization_matrix, model_dry)
            # 使用x_rainy_ind_grid, x_dry_ind_grid将dry和rainy的预测值填充到一个矩阵中
            area_data = combine_data(y_pre_inverse_grid_rainy,y_pre_inverse_grid_dry,x_rainy_ind_grid,x_dry_ind_grid)
            print(area_data[:,1,1])
            # 拼接到大矩阵中
            lon_size,lat_size = area_data.shape[1],area_data.shape[2]
            y_pre_inverse_grid[:,i*area_size:i*area_size+lon_size,j*area_size:j*area_size+lat_size] = area_data
# 保存y_pre_inverse_grid
file_path = r"C:\Users\Administrator\Desktop\code\LSTM\SMAP_ERA5_Data\all_exp\10_data"
np.save(os.path.join(file_path, "y_pre_inverse_grid.npy"), y_pre_inverse_grid)


C:\Users\Administrator\AppData\Local\Temp\ipykernel_54420\2381953606.py:83: RuntimeWarning: All-NaN slice encountered
  min_vals = np.nanmin(data, axis=0)
C:\Users\Administrator\AppData\Local\Temp\ipykernel_54420\2381953606.py:84: RuntimeWarning: All-NaN slice encountered
  max_vals = np.nanmax(data, axis=0)


35/35 [==============================] - 0s 7ms/step
[0.337759663415083 0.3368421871218814 0.33536462665783073 ...
 0.3359022518759378 0.33515099990671793 0.33324749136706977]


C:\Users\Administrator\AppData\Local\Temp\ipykernel_54420\2381953606.py:83: RuntimeWarning: All-NaN slice encountered
  min_vals = np.nanmin(data, axis=0)
C:\Users\Administrator\AppData\Local\Temp\ipykernel_54420\2381953606.py:84: RuntimeWarning: All-NaN slice encountered
  max_vals = np.nanmax(data, axis=0)


35/35 [==============================] - 0s 7ms/step
[0.15570395883314347 0.15913636516527419 0.1571437721935185 ...
 0.16168469538184738 0.15941379395226551 0.16271382021356562]


C:\Users\Administrator\AppData\Local\Temp\ipykernel_54420\2381953606.py:83: RuntimeWarning: All-NaN slice encountered
  min_vals = np.nanmin(data, axis=0)
C:\Users\Administrator\AppData\Local\Temp\ipykernel_54420\2381953606.py:84: RuntimeWarning: All-NaN slice encountered
  max_vals = np.nanmax(data, axis=0)


...

35/35 [==============================] - 0s 7ms/step
[0.11663742645581152 0.0872985343140602 0.0996566709673802 ...
 0.1050584469101776 0.10781627887523212 0.09103470692946414]
Error:lon_num or lat_num is too large


C:\Users\Administrator\AppData\Local\Temp\ipykernel_54420\2381953606.py:83: RuntimeWarning: All-NaN slice encountered
  min_vals = np.nanmin(data, axis=0)
C:\Users\Administrator\AppData\Local\Temp\ipykernel_54420\2381953606.py:84: RuntimeWarning: All-NaN slice encountered
  max_vals = np.nanmax(data, axis=0)


35/35 [==============================] - 0s 7ms/step
[0.13125187174770003 0.12901942949999867 0.19042647217680564 ...
 0.13230701712767567 0.1778396312544892 0.1316016620877396]
Error:lon_num or lat_num is too large


C:\Users\Administrator\AppData\Local\Temp\ipykernel_54420\2381953606.py:83: RuntimeWarning: All-NaN slice encountered
  min_vals = np.nanmin(data, axis=0)
C:\Users\Administrator\AppData\Local\Temp\ipykernel_54420\2381953606.py:84: RuntimeWarning: All-NaN slice encountered
  max_vals = np.nanmax(data, axis=0)


35/35 [==============================] - 0s 7ms/step
[0.12564502176787218 0.12646489495452173 0.12826287104924017 ...
 0.1251134889569685 0.1314315829623558 0.1280188541855789]


### <font face="宋体" size=4 color=#A52A2A> 时序实验

In [ ]:
target_var = "soil_moisture"
file_path = r"D:\Data_Store\combined_data\combined_SMAP_ERA5_ACTEMP_data.nc"
data = xr.open_dataset(file_path)

# 数据提取
data = data_preparation(file_path,target_var)
# Maqu	102.2	33.9 : 21,92;~~~~        Shiquanhe	80	32.5:   11,33;~~~~       Naqu	92	31.3:  14,65;~~~~
data = select_data(data,10,3,10) #提取的顺序是按经纬度顺序，经纬度为递增，因此为左下，Maqu：21 92:3,10  Shiquanhe：11 33:2,4  Naqu：14 65:2,7,写反了
data = data.sel(time=slice("2015-03-31", "2023-01-01"))
# 定义时间步长(2个月)
time_step = 60
timestamps = np.arange(data.time.shape[0])

# 定义单幅尺寸,研究区尺寸为area_size*area_size
area_size = 10
# 定义研究区域经纬度尺寸
lon_shape = data.lon.shape[0]
lat_shape = data.lat.shape[0]

lon_num = (lon_shape+1)//area_size
lat_num = (lat_shape+1)//area_size

# 数据分辨率
lon_res = 0.3734444444444444
lat_res = 0.3734449339207049

# 方法参数设置
methods_second = "MinMax"
methods_first = "rainydry"

# 变量列表
var_list = [
    "soil_moisture", "DEM",
    "Longitude", "Latitude", "DOY",
    # Era5-land hourly vars
    "dewpoint_temperature_2m","temperature_2m","skin_temperature",
    "soil_temperature_level_1","soil_temperature_level_2","soil_temperature_level_3","soil_temperature_level_4",
    "volumetric_soil_water_layer_1","volumetric_soil_water_layer_2","volumetric_soil_water_layer_3","volumetric_soil_water_layer_4",
    "surface_pressure",
    #Era5-land AC vars
    "total_precipitation","snowfall","snowmelt",
    "evaporation_from_bare_soil","evaporation_from_open_water_surfaces_excluding_oceans","evaporation_from_the_top_of_canopy","evaporation_from_vegetation_transpiration","total_evaporation",
    "runoff","sub_surface_runoff","surface_runoff",
    "surface_latent_heat_flux","surface_net_solar_radiation","surface_net_thermal_radiation",
]
missing_values = {
    "soil_moisture": -9999.0, 
    "DEM": -9999.0,
    "Longitude": -9999.0,
    "Latitude": -9999.0,
    "DOY": -9999.0,
    #Era5-land hourly vars
    "dewpoint_temperature_2m": -9999.0,
    "temperature_2m": -9999.0,
    "skin_temperature": -9999.0,
    "soil_temperature_level_1": -9999.0,
    "soil_temperature_level_2": -9999.0,
    "soil_temperature_level_3": -9999.0,
    "soil_temperature_level_4": -9999.0,
    "volumetric_soil_water_layer_1": -9999.0,
    "volumetric_soil_water_layer_2": -9999.0,
    "volumetric_soil_water_layer_3": -9999.0,
    "volumetric_soil_water_layer_4": -9999.0,
    "surface_pressure": -9999.0,
    #Era5-land AC vars
    "total_precipitation": -9999.0,
    "snowfall": -9999.0,
    "snowmelt": -9999.0,
    "evaporation_from_bare_soil": -9999.0,
    "evaporation_from_open_water_surfaces_excluding_oceans": -9999.0,
    "evaporation_from_the_top_of_canopy": -9999.0,
    "evaporation_from_vegetation_transpiration": -9999.0,
    "total_evaporation": -9999.0,
    "runoff": -9999.0,
    "sub_surface_runoff": -9999.0,
    "surface_runoff": -9999.0,
    "surface_latent_heat_flux": -9999.0,
    "surface_net_solar_radiation": -9999.0,
    "surface_net_thermal_radiation": -9999.0,
}

# 设置训练参数
TIME_STEPS = time_step
INPUT_DIMS = len(var_list)
lstm_units = 64
drop = 0.25

# 经纬度
lon_min = data.lon.min()
lon_max = data.lon.max()
lat_min = data.lat.min()
lat_max = data.lat.max()

In [ ]:
def find_nearest_index(array, value):
    array = np.asarray(array)
    idx = (np.abs(array - value)).argmin()
    return idx
lat_array = data.lat.values
lon_array = data.lon.values
lat_idx = find_nearest_index(lat_array, 33.9)
lon_idx = find_nearest_index(lon_array, 102.2)
print(lat_idx, lon_idx)

In [ ]:
# 下面所有的数组全都存在多个维度如rainy和dry
# 分为粗和细粒度分辨率
test_rmse,test_mae,test_r2 = np.empty((lon_num, lat_num,2), dtype=object),np.empty((lon_num, lat_num,2), dtype=object),np.empty((lon_num, lat_num,2), dtype=object)
train_rmse,train_mae,train_r2 = np.empty((lon_num, lat_num,2), dtype=object),np.empty((lon_num, lat_num,2), dtype=object),np.empty((lon_num, lat_num,2), dtype=object)
lon_shape_cor,lat_shape_cor = lon_num*area_size,lat_num*area_size
test_rmse_grid,test_mae_grid,test_r2_grid = np.empty((lon_shape_cor, lat_shape_cor,2), dtype=object),np.empty((lon_shape_cor, lat_shape_cor,2), dtype=object),np.empty((lon_shape_cor, lat_shape_cor,2), dtype=object)
train_rmse_grid,train_mae_grid,train_r2_grid = np.empty((lon_shape_cor, lat_shape_cor,2), dtype=object),np.empty((lon_shape_cor, lat_shape_cor,2), dtype=object),np.empty((lon_shape_cor, lat_shape_cor,2), dtype=object)

# 大区域样本预测值，大区域样本实际值
test_y_pred,test_y_true = np.empty((lon_shape_cor, lat_shape_cor,2), dtype=object),np.empty((lon_shape_cor, lat_shape_cor,2), dtype=object)
train_y_pred,train_y_true = np.empty((lon_shape_cor, lat_shape_cor,2), dtype=object),np.empty((lon_shape_cor, lat_shape_cor,2), dtype=object)

# 循环返回粗和细粒度分辨率矩阵用于绘制误差图
for i in range(lon_num):
    for j in range(lat_num):
        if i > (lon_shape//area_size) or j > (lat_shape//area_size):
            continue
        else:
            # 获取区域数据
            area_data = select_data(data,i+1,j+1,area_size)
            # 数据掩码和掩码矩阵生成
            masked_area_dataset,mask_array = mask_fun(area_data,missing_values,var_list)
            # 数据标准化
            MinMaxNormalization_data, MinMaxNormalization_matrix = MinMaxNormalization(masked_area_dataset)
            # 数据划分
            x_train_rainy, y_train_rainy, y_train_true_rainy, x_test_rainy, y_test_rainy, y_test_true_rainy,\
            x_train_dry, y_train_dry, y_train_true_dry, x_test_dry, y_test_dry, y_test_true_dry,\
            x_train_rainy_grid, x_test_rainy_grid, x_train_dry_grid, x_test_dry_grid,\
            y_true_train_rainy_grid, y_true_test_rainy_grid, y_true_train_dry_grid, y_true_test_dry_grid = \
            area_split_random_without_filling(area_data,area_size, MinMaxNormalization_data, MinMaxNormalization_matrix, timestamps, methods_first=methods_first, methods_second=methods_second, time_step=time_step)
            # 如不存在训练数据，跳过
            if x_train_rainy.shape[0] == 0 or x_train_dry.shape[0] == 0:
                continue
            # 训练模型
            model_rainy, history_rainy= train_model(x_train_rainy, y_train_rainy, time_step, x_train_rainy.shape[-1], drop)
            model_dry,  history_dry= train_model(x_train_dry, y_train_dry, time_step, x_train_dry.shape[-1], drop)
            # 保存模型
            file_model_path = r"C:\Users\Administrator\Desktop\code\LSTM\SMAP_ERA5_Data\surface_point_valid_exp\model"
            rainy_str = f"lon{i}_lat{j}_rainy"
            dry_str = f"lon{i}_lat{j}_dry"
            model_save(file_model_path, model_rainy, history_rainy, rainy_str)
            model_save(file_model_path, model_dry, history_dry, dry_str)

            # 区域预测
            y_test_pre_inverse_rainy, y_test_pre_inverse_grid_rainy = predict_area(area_data,area_size, x_test_rainy_grid,MinMaxNormalization_matrix, model_rainy)
            y_test_pre_inverse_dry, y_test_pre_inverse_grid_dry = predict_area(area_data,area_size, x_test_dry_grid, MinMaxNormalization_matrix, model_dry)
            y_train_pre_inverse_rainy, y_train_pre_inverse_grid_rainy = predict_area(area_data,area_size, x_train_rainy_grid, MinMaxNormalization_matrix, model_rainy)
            y_train_pre_inverse_dry, y_train_pre_inverse_grid_dry = predict_area(area_data,area_size, x_train_dry_grid, MinMaxNormalization_matrix, model_dry)

            # 评估
            test_rmse_rainy,test_r2_rainy,test_mae_rainy = evaluate(y_test_true_rainy, y_test_pre_inverse_rainy)
            test_rmse_dry,test_r2_dry,test_mae_dry = evaluate(y_test_true_dry, y_test_pre_inverse_dry)
            train_rmse_rainy, train_r2_rainy, train_mae_rainy = evaluate(y_train_true_rainy, y_train_pre_inverse_rainy)
            train_rmse_dry, train_r2_dry, train_mae_dry = evaluate(y_train_true_dry, y_train_pre_inverse_dry)

            # 逐个预测
            test_y_pred[i*area_size:(i+1)*area_size,j*area_size:(j+1)*area_size,:],train_y_pred[i*area_size:(i+1)*area_size,j*area_size:(j+1)*area_size,:] = np.stack((y_test_pre_inverse_grid_rainy,y_test_pre_inverse_grid_dry),axis=-1),np.stack((y_train_pre_inverse_grid_rainy,y_train_pre_inverse_grid_dry),axis=-1)
            test_y_true[i*area_size:(i+1)*area_size,j*area_size:(j+1)*area_size,:],train_y_true[i*area_size:(i+1)*area_size,j*area_size:(j+1)*area_size,:] = np.stack((y_true_test_rainy_grid,y_true_test_dry_grid),axis=-1),np.stack((y_true_train_rainy_grid,y_true_train_dry_grid),axis=-1)

            # 将grid数据以一块一块的形式拼接到大块中
            test_rmse_grid[i*area_size:(i+1)*area_size,j*area_size:(j+1)*area_size,:],test_mae_grid[i*area_size:(i+1)*area_size,j*area_size:(j+1)*area_size,:],test_r2_grid[i*area_size:(i+1)*area_size,j*area_size:(j+1)*area_size,:] = evaluate_grid(np.stack((y_true_test_rainy_grid,y_true_test_dry_grid),axis=2),np.stack((y_test_pre_inverse_grid_rainy,y_test_pre_inverse_grid_dry),axis=2))
            train_rmse_grid[i*area_size:(i+1)*area_size,j*area_size:(j+1)*area_size,:],train_mae_grid[i*area_size:(i+1)*area_size,j*area_size:(j+1)*area_size,:],train_r2_grid[i*area_size:(i+1)*area_size,j*area_size:(j+1)*area_size,:] = evaluate_grid(np.stack((y_true_train_rainy_grid,y_true_train_dry_grid),axis=2),np.stack((y_train_pre_inverse_grid_rainy,y_train_pre_inverse_grid_dry),axis=2))

            # 填充数据预测
            test_rmse[i,j,:],test_mae[i,j,:],test_r2[i,j,:] = np.stack((test_rmse_rainy,test_rmse_dry),axis=-1),np.stack((test_mae_rainy,test_mae_dry),axis=-1),np.stack((test_r2_rainy,test_r2_dry),axis=-1)
            train_rmse[i,j,:],train_mae[i,j,:],train_r2[i,j,:] = np.stack((train_rmse_rainy,train_rmse_dry),axis=-1),np.stack((train_mae_rainy,train_mae_dry),axis=-1),np.stack((train_r2_rainy,train_r2_dry),axis=-1)

# 初始化列表以收集所有点的数据
test_y_pred_all_rainy, test_y_true_all_rainy = [], []
test_y_pred_all_dry, test_y_true_all_dry = [], []
train_y_pred_all_rainy, train_y_true_all_rainy = [], []
train_y_pred_all_dry, train_y_true_all_dry = [], []

for i in range(lon_shape_cor):
    for j in range(lat_shape_cor):
        # 对于每个点，确保数据存在，然后添加到列表中
        if test_y_pred[i, j, 0] is not None and test_y_pred[i, j, 0].size != 0:
            test_y_pred_all_rainy.append(test_y_pred[i, j, 0])
            test_y_true_all_rainy.append(test_y_true[i, j, 0])
        if test_y_pred[i, j, 1] is not None and test_y_pred[i, j, 1].size != 0:
            test_y_pred_all_dry.append(test_y_pred[i, j, 1])
            test_y_true_all_dry.append(test_y_true[i, j, 1])

        if train_y_pred[i, j, 0] is not None and train_y_pred[i, j, 0].size != 0:
            train_y_pred_all_rainy.append(train_y_pred[i, j, 0])
            train_y_true_all_rainy.append(train_y_true[i, j, 0])
        if train_y_pred[i, j, 1] is not None and train_y_pred[i, j, 1].size != 0:
            train_y_pred_all_dry.append(train_y_pred[i, j, 1])
            train_y_true_all_dry.append(train_y_true[i, j, 1])

# 使用numpy.concatenate连接所有点的数据
test_y_pred_concat_rainy = np.concatenate(test_y_pred_all_rainy)
test_y_true_concat_rainy = np.concatenate(test_y_true_all_rainy)
test_y_pred_concat_dry = np.concatenate(test_y_pred_all_dry)
test_y_true_concat_dry = np.concatenate(test_y_true_all_dry)

train_y_pred_concat_rainy = np.concatenate(train_y_pred_all_rainy)
train_y_true_concat_rainy = np.concatenate(train_y_true_all_rainy)
train_y_pred_concat_dry = np.concatenate(train_y_pred_all_dry)
train_y_true_concat_dry = np.concatenate(train_y_true_all_dry)

# 计算全域内的误差
test_rmse_all_rainy, test_mae_all_rainy, test_r2_all_rainy = evaluate(test_y_true_concat_rainy, test_y_pred_concat_rainy)
test_rmse_all_dry, test_mae_all_dry, test_r2_all_dry = evaluate(test_y_true_concat_dry, test_y_pred_concat_dry)
train_rmse_all_rainy, train_mae_all_rainy, train_r2_all_rainy = evaluate(train_y_true_concat_rainy, train_y_pred_concat_rainy)
train_rmse_all_dry, train_mae_all_dry, train_r2_all_dry = evaluate(train_y_true_concat_dry, train_y_pred_concat_dry)

data_path = r"C:\Users\Administrator\Desktop\code\LSTM\SMAP_ERA5_Data\surface_point_valid_exp\data"
# 保存误差变量
save_list = [test_rmse, test_mae, test_r2, train_rmse, train_mae, train_r2, test_rmse_grid, test_mae_grid, test_r2_grid, train_rmse_grid, train_mae_grid, train_r2_grid,
             test_rmse_all_rainy, test_mae_all_rainy, test_r2_all_rainy, test_rmse_all_dry, test_mae_all_dry, test_r2_all_dry, train_rmse_all_rainy, train_mae_all_rainy, train_r2_all_rainy, train_rmse_all_dry, train_mae_all_dry, train_r2_all_dry]

variables_dict = {
    "test_rmse": test_rmse,"test_mae": test_mae,"test_r2": test_r2,"train_rmse": train_rmse,"train_mae": train_mae,"train_r2": train_r2,\
    "test_rmse_grid": test_rmse_grid,"test_mae_grid": test_mae_grid,"test_r2_grid": test_r2_grid,"train_rmse_grid": train_rmse_grid,"train_mae_grid": train_mae_grid,"train_r2_grid": train_r2_grid,\
    "test_rmse_all_rainy": test_rmse_all_rainy,"test_mae_all_rainy": test_mae_all_rainy,"test_r2_all_rainy": test_r2_all_rainy,"test_rmse_all_dry": test_rmse_all_dry,"test_mae_all_dry": test_mae_all_dry,"test_r2_all_dry": test_r2_all_dry,\
    "train_rmse_all_rainy": train_rmse_all_rainy,"train_mae_all_rainy": train_mae_all_rainy,"train_r2_all_rainy": train_r2_all_rainy,"train_rmse_all_dry": train_rmse_all_dry,"train_mae_all_dry": train_mae_all_dry,"train_r2_all_dry": train_r2_all_dry
}
# 所有以save_list变量名保存
for var_name, var_data in variables_dict.items():
    file_path = os.path.join(data_path, var_name + ".npy")
    np.save(file_path, var_data)


In [ ]:
# 区域预测
y_test_pre_inverse_rainy, y_test_pre_inverse_grid_rainy = predict_area(area_data,area_size, x_test_rainy_grid,MinMaxNormalization_matrix, model_rainy)
y_test_pre_inverse_dry, y_test_pre_inverse_grid_dry = predict_area(area_data,area_size, x_test_dry_grid, MinMaxNormalization_matrix, model_dry)
y_train_pre_inverse_rainy, y_train_pre_inverse_grid_rainy = predict_area(area_data,area_size, x_train_rainy_grid, MinMaxNormalization_matrix, model_rainy)
y_train_pre_inverse_dry, y_train_pre_inverse_grid_dry = predict_area(area_data,area_size, x_train_dry_grid, MinMaxNormalization_matrix, model_dry)

# 评估
test_rmse_rainy,test_r2_rainy,test_mae_rainy = evaluate(y_test_true_rainy, y_test_pre_inverse_rainy)
test_rmse_dry,test_r2_dry,test_mae_dry = evaluate(y_test_true_dry, y_test_pre_inverse_dry)
train_rmse_rainy, train_r2_rainy, train_mae_rainy = evaluate(y_train_true_rainy, y_train_pre_inverse_rainy)
train_rmse_dry, train_r2_dry, train_mae_dry = evaluate(y_train_true_dry, y_train_pre_inverse_dry)

# 逐个预测
test_y_pred[i*area_size:(i+1)*area_size,j*area_size:(j+1)*area_size,:],train_y_pred[i*area_size:(i+1)*area_size,j*area_size:(j+1)*area_size,:] = np.stack((y_test_pre_inverse_grid_rainy,y_test_pre_inverse_grid_dry),axis=-1),np.stack((y_train_pre_inverse_grid_rainy,y_train_pre_inverse_grid_dry),axis=-1)
test_y_true[i*area_size:(i+1)*area_size,j*area_size:(j+1)*area_size,:],train_y_true[i*area_size:(i+1)*area_size,j*area_size:(j+1)*area_size,:] = np.stack((y_true_test_rainy_grid,y_true_test_dry_grid),axis=-1),np.stack((y_true_train_rainy_grid,y_true_train_dry_grid),axis=-1)

# 将grid数据以一块一块的形式拼接到大块中
test_rmse_grid[i*area_size:(i+1)*area_size,j*area_size:(j+1)*area_size,:],test_mae_grid[i*area_size:(i+1)*area_size,j*area_size:(j+1)*area_size,:],test_r2_grid[i*area_size:(i+1)*area_size,j*area_size:(j+1)*area_size,:] = evaluate_grid(np.stack((y_true_test_rainy_grid,y_true_test_dry_grid),axis=2),np.stack((y_test_pre_inverse_grid_rainy,y_test_pre_inverse_grid_dry),axis=2))
train_rmse_grid[i*area_size:(i+1)*area_size,j*area_size:(j+1)*area_size,:],train_mae_grid[i*area_size:(i+1)*area_size,j*area_size:(j+1)*area_size,:],train_r2_grid[i*area_size:(i+1)*area_size,j*area_size:(j+1)*area_size,:] = evaluate_grid(np.stack((y_true_train_rainy_grid,y_true_train_dry_grid),axis=2),np.stack((y_train_pre_inverse_grid_rainy,y_train_pre_inverse_grid_dry),axis=2))

# 填充数据预测
test_rmse[i,j,:],test_mae[i,j,:],test_r2[i,j,:] = np.stack((test_rmse_rainy,test_rmse_dry),axis=-1),np.stack((test_mae_rainy,test_mae_dry),axis=-1),np.stack((test_r2_rainy,test_r2_dry),axis=-1)
train_rmse[i,j,:],train_mae[i,j,:],train_r2[i,j,:] = np.stack((train_rmse_rainy,train_rmse_dry),axis=-1),np.stack((train_mae_rainy,train_mae_dry),axis=-1),np.stack((train_r2_rainy,train_r2_dry),axis=-1)

#### <font face="宋体" size=3 color=#A52A

### <font face="宋体" size=4 color=#A52A2A>图像绘制

#### <font face="宋体" size=3 color=#A52A2A>误差图像绘制

In [ ]:
import rasterio
import numpy as np
import matplotlib.pyplot as plt
import cartopy.crs as ccrs
import cartopy.feature as cfeature
import cartopy.io.shapereader as shpreader
import xarray as xr
import geopandas as gpd
from matplotlib.path import Path
from matplotlib.patches import PathPatch
# 读取地理边界
shp_path = r"D:\Data_Store\TPBoundary_new(2021)\TPBoundary_new(2021).shp"
tp_boundary = gpd.read_file(shp_path)
def draw_err(data, lon_min, lon_max, lat_min, lat_max, area_size, lon_shape, lat_shape, method="thin"):
    # 计算粗和细粒度的数据形状
    lon_num = (lon_shape + 1) // area_size
    lat_num = (lat_shape + 1) // area_size
    lon_shape_cor, lat_shape_cor = lon_num * area_size, lat_num * area_size

    # 初始化画布
    fig = plt.figure(figsize=(12, 10))
    ax = plt.axes(projection=ccrs.PlateCarree())

    # 绘制地图特征
    ax.add_feature(cfeature.LAND, zorder=0, edgecolor='black', facecolor='lightgray') # 陆地
    ax.add_feature(cfeature.COASTLINE, zorder=1) # 海岸线
    ax.add_feature(cfeature.BORDERS, linestyle=':', zorder=1) # 国界线

    # 设置地图范围
    ax.set_extent([lon_min, lon_max, lat_min, lat_max], crs=ccrs.PlateCarree())

    # 细粒度绘制
    if method == "thin":
        data_full = data[:lon_shape, :lat_shape]
        lon_edges = np.linspace(lon_min, lon_max, data_full.shape[1] + 1)  # 这将创建100个边界值
        lat_edges = np.linspace(lat_min, lat_max, data_full.shape[0] + 1)  # 这将创建30个边界值
        lon, lat = np.meshgrid(lon_edges, lat_edges)
        mesh = ax.pcolormesh(lon, lat, data_full, transform=ccrs.PlateCarree(), shading='auto')

    # 粗粒度绘制
    elif method == "thick":
        # 先对数据进行扩展
        data_expanded = np.repeat(np.repeat(data, area_size, axis=0), area_size, axis=1)
        data_full = data_expanded[:lon_shape_cor, :lat_shape_cor, :]
        lon, lat = np.meshgrid(np.linspace(lon_min, lon_max, lon_shape_cor), np.linspace(lat_min, lat_max, lat_shape_cor))
        mesh = ax.pcolormesh(lon, lat, data_full[:, :], transform=ccrs.PlateCarree(), shading='auto',zorder=2)  # 同样示意用第一个误差数据绘图
    
    # 创建青藏高原边界路径
    boundary_path = Path(np.concatenate([np.array(geom.exterior.coords) for geom in tp_boundary.geometry])) # 读取边界数据
    clip_path = PathPatch(boundary_path, transform=ccrs.PlateCarree()._as_mpl_transform(ax), facecolor='lightgray', edgecolor='black', zorder=0)
    ax.add_patch(clip_path)
    # 使用路径裁剪矩形补丁反转掩码效果 
    ax.add_patch(clip_path) # 添加裁剪补丁

    # 将颜色网格（mesh）裁剪到青藏高原边界内
    mesh.set_clip_path(clip_path) # 裁剪网格

    # 添加颜色条
    plt.colorbar(mesh, ax=ax, shrink=0.5)

    # 添加经纬度网格线
    ax.gridlines(draw_labels=True, dms=True, x_inline=False, y_inline=False)

    # 添加标题
    plt.title('Error Map ({})'.format(method.title()))

    # 显示图像
    plt.show()

draw_err(np.array(test_r2_grid[:,:,0],dtype='float'), lon_min, lon_max, lat_min, lat_max, area_size, lon_shape, 29, method="thin")


#### <font face="宋体" size=3 color=#A52A2A>单点时序图绘制
- <font face="宋体" size=2 color=#A52A2A>数据准备：
    1. Maqu,shiquanhe两地原位数据准备，原位时间跨度为15-2019/9/8
    2.
- <font face="宋体" size=2 color=#A52A2A>绘制：
    1. 原始数据-以点的形式绘制，点的符号为黑色圈圈
    2. 表面测量数据-以线的形式绘制，线的符号为红色实线
    3. 预测数据-以点的形式绘制，点的符号为蓝色+号
    4. SMAPL4数据-以点的形式绘制，点的符号为绿色*号
    5. ERA5-land数据-以点的形式绘制，点的符号为橙色*号
- <font face="宋体" size=2 color=#A52A2A>绘制结构：
    1. 设置画布
    2. 线绘制函数，传入数据和线条符号、线宽、颜色
    3. 点绘制函数，传入数据和点符号、点大、颜色
    4. 绘制其他修饰部分：时间刻度处理、雨季旱季分界虚线和不同背景设置、y轴上下限设置

In [ ]:
data
'''
var_list= [
    "soil_moisture","DEM",
    "Longitude", "Latitude", "DOY",
    #Era5-land hourly vars
    "dewpoint_temperature_2m","temperature_2m","skin_temperature",
    "soil_temperature_level_1","soil_temperature_level_2","soil_temperature_level_3","soil_temperature_level_4",
    "volumetric_soil_water_layer_1","volumetric_soil_water_layer_2","volumetric_soil_water_layer_3","volumetric_soil_water_layer_4",
    "surface_pressure",
    #Era5-land AC vars
    "total_precipitation","snowfall","snowmelt",
    "evaporation_from_bare_soil","evaporation_from_open_water_surfaces_excluding_oceans","evaporation_from_the_top_of_canopy","evaporation_from_vegetation_transpiration","total_evaporation",
    "runoff","sub_surface_runoff","surface_runoff",
    "surface_latent_heat_flux","surface_net_solar_radiation","surface_net_thermal_radiation",
]
'''

'''
#var_list = [
    "soil_moisture", "DEM",
    "Longitude", "Latitude", "DOY",
    # Era5-land hourly vars
    "dewpoint_temperature_2m","temperature_2m","skin_temperature",
    "soil_temperature_level_1","soil_temperature_level_2","soil_temperature_level_3","soil_temperature_level_4",
    "volumetric_soil_water_layer_1","volumetric_soil_water_layer_2","volumetric_soil_water_layer_3","volumetric_soil_water_layer_4",
    "surface_pressure",
    #Era5-land AC vars
    "total_precipitation","snowfall","snowmelt",
    "evaporation_from_bare_soil","evaporation_from_open_water_surfaces_excluding_oceans","evaporation_from_the_top_of_canopy","evaporation_from_vegetation_transpiration","total_evaporation",
    "runoff","sub_surface_runoff","surface_runoff",
    "surface_latent_heat_flux","surface_net_solar_radiation","surface_net_thermal_radiation",
]
#missing_values = {
    "soil_moisture": -9999.0, 
    "DEM": -9999.0,
    "Longitude": -9999.0,
    "Latitude": -9999.0,
    "DOY": -9999.0,
    #Era5-land hourly vars
    "dewpoint_temperature_2m": -9999.0,
    "temperature_2m": -9999.0,
    "skin_temperature": -9999.0,
    "soil_temperature_level_1": -9999.0,
    "soil_temperature_level_2": -9999.0,
    "soil_temperature_level_3": -9999.0,
    "soil_temperature_level_4": -9999.0,
    "volumetric_soil_water_layer_1": -9999.0,
    "volumetric_soil_water_layer_2": -9999.0,
    "volumetric_soil_water_layer_3": -9999.0,
    "volumetric_soil_water_layer_4": -9999.0,
    "surface_pressure": -9999.0,
    #Era5-land AC vars
    "total_precipitation": -9999.0,
    "snowfall": -9999.0,
    "snowmelt": -9999.0,
    "evaporation_from_bare_soil": -9999.0,
    "evaporation_from_open_water_surfaces_excluding_oceans": -9999.0,
    "evaporation_from_the_top_of_canopy": -9999.0,
    "evaporation_from_vegetation_transpiration": -9999.0,
    "total_evaporation": -9999.0,
    "runoff": -9999.0,
    "sub_surface_runoff": -9999.0,
    "surface_runoff": -9999.0,
    "surface_latent_heat_flux": -9999.0,
    "surface_net_solar_radiation": -9999.0,
    "surface_net_thermal_radiation": -9999.0,
}
'''